<a href="https://colab.research.google.com/github/batust/presidio-db-json/blob/main/pre-prcoessor-data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install presidio_analyzer
!pip install presidio_anonymizer
!python -m spacy download en_core_web_lg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 4.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
# Trying out batch processing
from typing import List, Optional, Dict, Union, Iterator, Iterable
import collections
from dataclasses import dataclass
import pprint

import pandas as pd

from presidio_analyzer import AnalyzerEngine, BatchAnalyzerEngine, RecognizerResult, DictAnalyzerResult
from presidio_anonymizer import AnonymizerEngine, BatchAnonymizerEngine
from presidio_anonymizer.entities import EngineResult

In [ ]:
# Authenicating and accessing Llama
from os import environ
from huggingface_hub import login

HUGGING_FACE_TOKEN = environ.get("HUGGING_FACE_TOKEN")
login(token=HUGGING_FACE_TOKEN)

In [ ]:
# Keeps crashing session
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
llama_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

def classify_sensitive_columns(columns):
    prompt = f"Given the column names {columns}, identify which may contain sensitive information (e.g., names, IDs, financial data). Return only a list of sensitive column names."

    response = llama_pipeline(prompt, max_length=100, do_sample=True)

    return response[0]['generated_text']

columns = ["Account", "Project ID", "Program Name", "Employee", "Cost Category",
           "Entity", "COA ID", "COA", "Day", "Rate", "QUANTITY", "Actuals",
           "Posting Category", "Description"]

print(classify_sensitive_columns(columns))

In [ ]:
!pip install flair
!pip install presidio_analyzer
!pip install presidio_anonymizer
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 15.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.6/139.6 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.6/202.6 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import files

uploaded = files.upload()
file_name = list(uploaded.keys())[0].strip().replace(' ', '-')

Saving testing-csv-ff.csv to testing-csv-ff.csv


In [ ]:
import pandas as pd
from flair.data import Sentence
from flair.models import SequenceTagger

EXCLUDED_HEADERS = ["description", "comment", "comments", "query", "notes", "email"]

# ner_tagger = SequenceTagger.load("flair/ner-english")
tagger = SequenceTagger.load('flair/pos-english')
if file_name.endswith('.xlsx'):
  df = pd.read_excel(file_name)
elif file_name.endswith('.csv'):
  df = pd.read_csv(file_name)

df

2025-03-19 11:28:24,203 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>
2025-03-19 11:28:25,113 SequenceTagger predicts: Dictionary with 53 tags: <unk>, O, UH, ,, VBD, PRP, VB, PRP$, NN, RB, ., DT, JJ, VBP, VBG, IN, CD, NNS, NNP, WRB, VBZ, WDT, CC, TO, MD, VBN, WP, :, RP, EX, JJR, FW, XX, HYPH, POS, RBR, JJS, PDT, NNPS, RBS, AFX, WP$, -LRB-, -RRB-, ``, '', LS, $, SYM, ADD


,id,name,domain,year founded,industry,size range,locality,country,linkedin url,current employee estimate,total employee estimate
0,5872184,ibm,ibm.com,1911.0,information technology and services,10001+,"new york, new york, united states",united states,linkedin.com/company/ibm,274047,716906
1,4425416,tata consultancy services,tcs.com,1968.0,information technology and services,10001+,"bombay, maharashtra, india",india,linkedin.com/company/tata-consultancy-services,190771,341369
2,21074,accenture,accenture.com,1989.0,information technology and services,10001+,"dublin, dublin, ireland",ireland,linkedin.com/company/accenture,190689,455768
3,2309813,us army,goarmy.com,1800.0,military,10001+,"alexandria, virginia, united states",united states,linkedin.com/company/us-army,162163,445958
4,1558607,ey,ey.com,1989.0,accounting,10001+,"london, greater london, united kingdom",united kingdom,linkedin.com/company/ernstandyoung,158363,428960
...,...,...,...,...,...,...,...,...,...,...,...
29995,1201877,hans anders,hansanders.nl,1982.0,retail,1001 - 5000,"giethoorn, overijssel, netherlands",netherlands,linkedin.com/company/hans-anders,390,959
29996,2828041,real canadian superstore,superstore.ca,NaN,retail,1001 - 5000,"madrid, madrid, spain",spain,linkedin.com/company/real-canadian-superstore,390,1320
29997,835865,"shumaker, loop & kendrick, llp",slk-law.com,1925.0,law practice,501 - 1000,"toledo, ohio, united states",united states,linkedin.com/company/shumaker-loop-&-kendrick-llp,390,688
29998,1611480,business wire,businesswire.com,1961.0,public relations and communications,501 - 1000,"san francisco, california, united states",united states,linkedin.com/company/business-wire,390,998


In [ ]:
# Not batch processing
def classify_cell_as_noun(texts):
    sentences = [Sentence(text) for text in texts]
    tagger.predict(sentences)

    results = []
    noun_count = 0

    for sentence, original_text in zip(sentences, texts):
        all_nouns = any("NN" in token.tag for token in sentence)
        classification = "Noun" if all_nouns else "Not a noun"

        results.append({"Original Text": original_text, "Classification": classification})
        noun_count += all_nouns

    is_noun_column = noun_count > (len(results) // 1.5)
    return results, is_noun_column

sensitive = set()

for column in df.columns:
    values = df[column].astype(str).tolist()
    results, is_noun = classify_cell_as_noun(values)

    if is_noun: sensitive.add(column)

print(sensitive)


In [ ]:
# Batch processing
import pandas as pd
import concurrent.futures
from flair.data import Sentence
from flair.models import SequenceTagger

tagger = SequenceTagger.load("flair/pos-english")

EXCLUDED_HEADERS = ["description", "comment", "comments", "query", "notes", "email"]
FILE_NAME = "companies_sorted.csv"
BATCH_SIZE = 10000
FLAIR_BATCH_SIZE = 100
MAX_THREADS = 4

sensitive = set()

def classify_cells_as_nouns(texts):
    """Batch processes text to check if columns are noun-heavy."""
    texts = ['' if pd.isna(text) else text for text in texts]

    results = []
    noun_count = 0

    for i in range(0, len(texts), FLAIR_BATCH_SIZE):
        batch_text = texts[i : i + FLAIR_BATCH_SIZE]
        sentences = [Sentence(text) for text in batch_text]

        tagger.predict(sentences)

        for sentence, original_text in zip(sentences, batch_text):
            all_nouns = any("NN" in token.tag for token in sentence)
            classification = "Noun" if all_nouns else "Not a noun"

            results.append({"Original Text": original_text, "Classification": classification})

            noun_count += all_nouns

    is_noun_column = noun_count > (len(results) * 0.66)

    return is_noun_column

def process_column(col, chunk):
    """Runs noun classification on a single column."""
    values = chunk[col].astype(str).fillna("").tolist()
    return col if classify_cells_as_nouns(values) else None

FILE_NAME = 'smaller_companies.csv'
if FILE_NAME.endswith(".xlsx"):
    df_iterator = pd.read_excel(FILE_NAME, chunksize=BATCH_SIZE)
elif FILE_NAME.endswith(".csv"):
    df_iterator = pd.read_csv(FILE_NAME, chunksize=BATCH_SIZE)

for chunk in df_iterator:
    with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_THREADS) as executor:
        futures = {executor.submit(process_column, col, chunk): col for col in chunk.columns}

        for future in concurrent.futures.as_completed(futures):
            result = future.result()
            if result:
                sensitive.add(result)

print("Sensitive Columns:", sensitive)

2025-03-19 11:51:26,893 SequenceTagger predicts: Dictionary with 53 tags: <unk>, O, UH, ,, VBD, PRP, VB, PRP$, NN, RB, ., DT, JJ, VBP, VBG, IN, CD, NNS, NNP, WRB, VBZ, WDT, CC, TO, MD, VBN, WP, :, RP, EX, JJR, FW, XX, HYPH, POS, RBR, JJS, PDT, NNPS, RBS, AFX, WP$, -LRB-, -RRB-, ``, '', LS, $, SYM, ADD


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:1124: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at /pytorch/aten/src/ATen/native/cudnn/RNN.cpp:1412.)
  result = _VF.lstm(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:1124: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at /pytorch/aten/src/ATen/native/cudnn/RNN.cpp:1412.)
  result = _VF.lstm(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:1124: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, pos

Sensitive Columns: {'linkedin url', 'name', 'country', 'industry', 'locality'}


In [ ]:
masked_df = df_iterator.copy()

for col in masked_df:
  if col in sensitive:
    masked_df[col] = [f'{col}_{i}' for i in range(len(masked_df[col]))]

masked_df

In [ ]:
# Trying spacy and flair on large dataset using Threading
import re
import json
import spacy
import pandas as pd
from flair.data import Sentence
from flair.models import SequenceTagger
from concurrent.futures import ThreadPoolExecutor
from threading import Lock

# Load models
flair_tagger = SequenceTagger.load("flair/pos-english")
nlp = spacy.load("en_core_web_sm")

# Constants
BATCH_SIZE = 10000  # Process in chunks
FLAIR_SAMPLE_SIZE = 10  # Sample size for noun detection
THREADS = 4  # Number of parallel threads
PHONE_REGEX = r"\b(\+\d{1,4}\s?)?(\(?\d{1,4}\)?[-.]?\s?)?\d{3,4}[-.]?\d{3,4}[-.]?\d{3,4}\b"

DESCRIPTIVE_COLUMNS = {"description", "comment", "comments", "query", "notes"}
SENSITIVE_COLUMNS = set()
MAPPINGS = {}
VALUE_TO_MASK = {}
ENTITY_COUNTER = 1
mask_lock = Lock()  # Thread-safe lock for dictionary updates

def is_noun_column(texts):
    """Identify if a column contains mostly nouns based on a sample."""
    texts = ["" if pd.isna(text) else str(text) for text in texts[:FLAIR_SAMPLE_SIZE]]
    sentences = [Sentence(text) for text in texts]
    flair_tagger.predict(sentences)
    noun_count = sum(any("NN" in token.tag for token in sent) for sent in sentences)
    return noun_count > (len(texts) * 0.6)

def mask_text(text, col_name, row_idx):
    """Mask the input text and store the mapping, ensuring thread-safe access."""
    global ENTITY_COUNTER
    if pd.isna(text) or str(text).strip() == "":
        return text
    with mask_lock:  # Thread-safe updates
        if text in VALUE_TO_MASK:
            return VALUE_TO_MASK[text]
        masked_value = f"{col_name}_{row_idx}"
        VALUE_TO_MASK[text] = masked_value
        MAPPINGS[masked_value] = text
        ENTITY_COUNTER += 1
    return masked_value

def process_chunk(chunk):
    """Process a chunk of the CSV, masking sensitive data, and return a new masked DataFrame."""
    new_chunk = chunk.copy()  # Ensure original DataFrame remains unchanged

    for row_idx in new_chunk.index:
        # Mask sensitive columns
        for col in SENSITIVE_COLUMNS:
            new_chunk.at[row_idx, col] = mask_text(new_chunk.at[row_idx, col], col, row_idx)

        # Mask phone numbers across all columns
        for col in new_chunk.columns:
            new_chunk.at[row_idx, col] = re.sub(PHONE_REGEX, lambda m: mask_text(m.group(0), "phone", row_idx), str(new_chunk.at[row_idx, col]))

        # Mask descriptive text columns
        descriptive_cols = [col for col in new_chunk.columns if col.lower() in DESCRIPTIVE_COLUMNS]
        for col in descriptive_cols:
            doc = nlp(str(new_chunk.at[row_idx, col]))
            for ent in doc.ents:
                new_chunk.at[row_idx, col] = new_chunk.at[row_idx, col].replace(ent.text, mask_text(ent.text, col, row_idx))

    return new_chunk  # Return masked DataFrame

def main(file_path):
    """Process the entire CSV file in chunks and write masked data to a new file."""
    global SENSITIVE_COLUMNS

    # Identify sensitive columns from a sample
    df_sample = pd.read_csv(file_path, nrows=FLAIR_SAMPLE_SIZE)
    SENSITIVE_COLUMNS = {col for col in df_sample.columns if is_noun_column(df_sample[col].astype(str).tolist())}

    # Process CSV in chunks with threading
    reader = pd.read_csv(file_path, chunksize=BATCH_SIZE)
    first_chunk = True  # Ensure header is written only once

    with ThreadPoolExecutor(max_workers=THREADS) as executor:
        for masked_chunk in executor.map(process_chunk, reader):
            # Append masked chunk to file (efficient memory usage)
            masked_chunk.to_csv("masked_output.csv", index=False, mode="a", header=first_chunk)
            first_chunk = False  # Disable header for subsequent chunks

    # Save mappings separately for possible de-anonymization
    with open("mappings.json", "w") as f:
        json.dump(MAPPINGS, f)

    print("Processing complete. Masked data saved.")

if __name__ == "__main__":
    main('smaller_companies.csv')

2025-03-20 08:47:18,085 SequenceTagger predicts: Dictionary with 53 tags: <unk>, O, UH, ,, VBD, PRP, VB, PRP$, NN, RB, ., DT, JJ, VBP, VBG, IN, CD, NNS, NNP, WRB, VBZ, WDT, CC, TO, MD, VBN, WP, :, RP, EX, JJR, FW, XX, HYPH, POS, RBR, JJS, PDT, NNPS, RBS, AFX, WP$, -LRB-, -RRB-, ``, '', LS, $, SYM, ADD


<ipython-input-24-72a88127c458>:61: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '5872184' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  new_chunk.at[row_idx, col] = re.sub(PHONE_REGEX, lambda m: mask_text(m.group(0), "phone", row_idx), str(new_chunk.at[row_idx, col]))
<ipython-input-24-72a88127c458>:61: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1911.0' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  new_chunk.at[row_idx, col] = re.sub(PHONE_REGEX, lambda m: mask_text(m.group(0), "phone", row_idx), str(new_chunk.at[row_idx, col]))
<ipython-input-24-72a88127c458>:61: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '274047' has dtype incompatible with int64,

Processing complete. Masked data saved.


In [ ]:
!pip install flair
!pip install presidio_analyzer
!pip install presidio_anonymizer
# !python -m spacy download en_core_web_lg
!pip install --upgrade --force-reinstall "numpy==1.26.4"
!pip install --upgrade --force-reinstall "Cython" "spacy"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.6/139.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.6/202.6 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Testing on smaller dataset
import re
import json
import spacy
import pandas as pd
from flair.data import Sentence
from flair.models import SequenceTagger
from threading import Lock

# Load models
flair_tagger = SequenceTagger.load("flair/pos-english")
nlp = spacy.load("en_core_web_sm")

# Constants
PHONE_REGEX = r"\b(\+\d{1,4}\s?)?(\(?\d{1,4}\)?[-.]?\s?)?\d{3,4}[-.]?\d{3,4}[-.]?\d{3,4}\b"
DESCRIPTIVE_COLUMNS = {"description", "comment", "comments", "query", "notes"}
SENSITIVE_COLUMNS = set()
MAPPINGS = {}
VALUE_TO_MASK = {}
mask_lock = Lock()

def is_noun_column(texts):
    """Identify if a column contains mostly nouns based on a sample."""
    texts = ["" if pd.isna(text) else str(text) for text in texts]
    sentences = [Sentence(text) for text in texts]
    flair_tagger.predict(sentences)
    noun_count = sum(any("NN" in token.tag for token in sent) for sent in sentences)
    return noun_count > (len(texts) * 0.6)

def mask_text(text, col_name, row_idx):
    """Mask the input text and store the mapping."""
    if pd.isna(text) or str(text).strip() == "":
        return text
    with mask_lock:
        if text in VALUE_TO_MASK:
            return VALUE_TO_MASK[text]
        masked_value = f"{col_name}_{row_idx}"
        VALUE_TO_MASK[text] = masked_value
        MAPPINGS[masked_value] = text
    return masked_value

def process_chunk(chunk):
    """Process a dataframe chunk (30 rows) for masking and return a new masked DataFrame."""
    # Create a copy of the DataFrame to avoid modifying the original
    new_chunk = chunk.copy()

    for row_idx in new_chunk.index:
        # Mask sensitive columns
        for col in SENSITIVE_COLUMNS:
            new_chunk.loc[row_idx, col] = mask_text(new_chunk.loc[row_idx, col], col, row_idx)

        # Mask phone numbers
        for col in new_chunk.columns:
            new_chunk.loc[row_idx, col] = re.sub(PHONE_REGEX, lambda m: mask_text(m.group(0), "phone", row_idx), str(new_chunk.loc[row_idx, col]))

        # Mask descriptive text columns
        for col in set(DESCRIPTIVE_COLUMNS) & set(new_chunk.columns):
            doc = nlp(str(new_chunk.loc[row_idx, col]))
            for ent in doc.ents:
                new_chunk.loc[row_idx, col] = new_chunk.loc[row_idx, col].replace(ent.text, mask_text(ent.text, col, row_idx))

    return new_chunk  # Returning the new modified DataFrame


def main(file_path):
    """Process only 30 rows of the CSV file."""
    global SENSITIVE_COLUMNS
    # Read first 30 rows
    df = pd.read_csv(file_path, nrows=30)

    # Identify sensitive columns (case-insensitive)
    SENSITIVE_COLUMNS = {col for col in df.columns if is_noun_column(df[col].astype(str).tolist())}

    # Process and save output
    processed_df = process_chunk(df)
    print("\nProcessed Data:\n", processed_df.head(10))  # Display first 10 rows for verification
    processed_df.to_csv("masked_output.csv", index=False)

    with open("mappings.json", "w") as f:
        json.dump(MAPPINGS, f)

    print("Processing complete. Masked data saved.")

if __name__ == "__main__":
    main('smaller_companies.csv')


# **TESTING CODE!**

In [ ]:
import re
import spacy

nlp = spacy.load("en_core_web_sm")

# Regex
EMAIL_REGEX = r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}"
PHONE_REGEX = r"\b(\+\d{1,4}\s?)?(\(?\d{1,4}\)?[-.]?\s?)?\d{3,4}[-.]?\d{3,4}[-.]?\d{3,4}\b"

# def detect_noun_desc(text):
#     doc = nlp(text)
#     modified_text = []

#     for token in doc:
#         token_text = token.text

#         if re.fullmatch(EMAIL_REGEX, token_text):
#             modified_text.append("<email>")
#         elif re.fullmatch(PHONE_REGEX, token_text):
#             modified_text.append("<phone>")
#         elif token.pos_ == "PROPN":
#             modified_text.append("<sensitive>")
#         else:
#             modified_text.append(token_text)

#     return " ".join(modified_text)

def detect_noun(file_path):
    if file_path.endswith(".csv"):
        df = pd.read_csv(file_path, engine="python")
    elif file_path.endswith((".xls", ".xlsx")):
        df = pd.read_excel(file_path)
    else:
        raise ValueError("Unsupported file format. Please provide a CSV or Excel file.")

    sensitive = []

    for col in df.columns:
        if df[col].dtype in ['int64', 'float64']:
            continue  # Skip purely numeric columns

        text_samples = df[col].astype(str).head(5)  # Take first 5 values
        noun_count = 0
        total_count = 0

        for value in text_samples:
            if "%" in value or value.replace(".", "").isdigit():
                continue  # Skip percentage or number-like values

            doc = nlp(value)
            for token in doc:
                if token.pos_ in ['NOUN', 'PROPN']:
                    noun_count += 1
                total_count += 1

        # Mark column as sensitive only if a significant portion are nouns
        if total_count > 0 and (noun_count / total_count) > 0.2:
            sensitive.append(col)

    return sensitive

In [ ]:
import pandas as pd
import re

def descriptive_cols(file_path):
    """
    Reads a CSV or Excel file, removes column names similar to 'description', 'remarks', etc.,
    and returns a filtered list of sensitive columns.
    """
    # Define keywords to filter out
    keywords = ["description", "remarks", "notes", "comments", "observations", "details", "summary", "explanation",
    "reviews", "feedback", "testimonials", "opinions", "assessment", "suggestions", "experience",
    "incident_report", "case_notes", "audit_notes", "findings", "status_update", "history", "progress_report",
    "additional_info", "clarifications", "justification", "annotations", "excerpts", "statement", "explanation_text"]

    # Ensure columns are properly loaded from CSV/Excel
    if file_path.endswith(".csv"):
        df = pd.read_csv(file_path, nrows=1)  # Read only header
    elif file_path.endswith((".xls", ".xlsx")):
        df = pd.read_excel(file_path, nrows=1, engine="openpyxl")  # Read only header
    else:
        raise ValueError("Unsupported file format. Please provide a CSV or Excel file.")

    # Get actual column names
    all_columns = df.columns.tolist()

    # Filter out columns that match the keywords
    filtered_columns = [col for col in all_columns if any(re.search(keyword, col, re.IGNORECASE) for keyword in keywords)]

    return filtered_columns




In [ ]:
import pandas as pd
import spacy

# nlp = spacy.load("en_core_web_sm")

def detect_noun(file_path):
    # Detect file type and read accordingly
    if file_path.endswith(".csv"):
        df = pd.read_csv(file_path, engine="python")
    elif file_path.endswith((".xls", ".xlsx")):
        df = pd.read_excel(file_path)
    else:
        raise ValueError("Unsupported file format. Please provide a CSV or Excel file.")

    sensitive = []
    column=[]
    for col in df.columns:
        column.append(col)
        # **Skip numeric columns**
        if pd.api.types.is_numeric_dtype(df[col]): continue

        # **Process only non-empty, valid string values**
        valid_values = df[col].dropna().astype(str)  # Remove NaN
        valid_values = [val for val in valid_values.head(3) if val.lower() not in ["nan", "none", "","na","null","n/a"]]

        for value in valid_values:
            doc = nlp(value)
            noun_count = sum(1 for token in doc if token.pos_ in ['NOUN', 'PROPN'])
            word_count = len(doc)

            # **Mark column as sensitive only if a high % of words are nouns**
            if word_count > 0 and (noun_count / word_count) > 0.2:
                if col not in sensitive:
                    sensitive.append(col)
    print(column)
    print(sensitive)
    return sensitive



In [ ]:
# Testing code!
# Main code
import json
import pandas as pd

mapping = {}

def csv_an(input_file, output_file):
    df = pd.read_csv(input_file, engine="python")

    sensitive_old = detect_noun(input_file)
    descrp = descriptive_cols(input_file)
    sensitive = list(set(sensitive_old) - set(descrp))
    column_counters = {col: 1 for col in df.columns if col in sensitive_old}
    column_mappings = {col: {} for col in sensitive}  # Store mappings for each column

    for col in sensitive:
        new_values = []
        for val in df[col].astype(str):
            if pd.notna(val):
                if val in column_mappings[col]:
                    anonymized_value = column_mappings[col][val]  # Use existing mapping
                else:
                    anonymized_value = f"{col}{column_counters[col]}"
                    column_mappings[col][val] = anonymized_value  # Store new mapping
                    column_counters[col] += 1  # Increment counter

                mapping[anonymized_value] = val
                new_values.append(anonymized_value)
            else:
                new_values.append(val)
        df[col] = new_values

    for col in descrp:
        for idx, val in enumerate(df[col].astype(str)):
            if pd.notna(val):
                df.at[idx, col] = detect_noun_desc(val)

    df.to_csv(output_file, index=False)
    print(f"Anonymized file saved as {output_file}")

    # Save mapping as JSON
    with open("mappings.json", "w") as f:
        json.dump(mapping, f)


csv_an('testing-csv-ff.csv', 'masked_csv_output.csv')


['Reporting Month', 'Cluster Name', 'Input Account Name', 'Account Name', 'New Assets Issuance', 'Allocations', 'Welcome Email', 'Onboarding Trainings', 'BGV', 'NDA', 'Granting Access', 'Client Onboarding', 'Deallocation', 'Assets Return', 'Access Removal', 'Compliance % on mandatory trainings', 'Compliance % on mandatory audits', 'Timesheet Compliance %']
['Reporting Month', 'Cluster Name', 'Input Account Name', 'Account Name', 'Client Onboarding', 'Compliance % on mandatory trainings', 'Compliance % on mandatory audits', 'Timesheet Compliance %']
Anonymized file saved as masked_csv_output.csv


# **CONSOLITATING CODE**

In [ ]:
!pip install flair
!pip install faker
!pip install presidio_analyzer
!pip install presidio_anonymizer
# !python -m spacy download en_core_web_lg
!pip install --upgrade --force-reinstall "numpy==1.26.4"
!pip install --upgrade --force-reinstall "Cython" "spacy"

  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.4
    Uninstalling numpy-2.2.4:
      Successfully uninstalled numpy-2.2.4
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Using cached Cython-3.0.12-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.3 kB)
  Using cached spacy-3.8.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (27 kB)
  Using cached spacy_legacy-3.0.12-py2.py3-none-any.whl.metadata (2.8 kB)
  Using cached spacy

In [ ]:
import re
import json
import time
import spacy
import openpyxl
import pandas as pd
from google.colab import files
mapping={}
nlp = spacy.load("en_core_web_sm")

def time_it(func):
  def wrapper(*args, **kwargs):
    start = time.time()
    result = func(*args, **kwargs)
    end = time.time()
    print(f'\n\nExecution time {func.__name__}: {end-start:.6f} seconds')
    return result
  return wrapper

#-------------------------------------------------------------------------------------------------------------------------------
#function to find out the noun values dominating columns considering it will be a sensitive data if there is so many nouns
#------------------------------------------------------------------------------------------------------------------------------
def detect_noun(file_path):
    if file_path.endswith(".csv"):
        df = pd.read_csv(file_path, engine="python")
    elif file_path.endswith((".xls", ".xlsx")):
        df = pd.read_excel(file_path)
    else:
        raise ValueError("Unsupported file format. Please provide a CSV or Excel file.")

    sensitive = []

    for col in df.columns:
        if df[col].dtype in ['int64', 'float64']:
            continue  # Skip purely numeric columns

        text_samples = df[col].astype(str).head(5)  # Take first 5 values
        noun_count = 0
        total_count = 0

        for value in text_samples:
            if "%" in value or value.replace(".", "").isdigit():
                continue  # Skip percentage or number-like values

            doc = nlp(value)
            for token in doc:
                if token.pos_ in ['NOUN', 'PROPN']:
                    noun_count += 1
                total_count += 1

        # Mark column as sensitive only if a significant portion are nouns
        if total_count > 0 and (noun_count / total_count) > 0.2:
            sensitive.append(col)

    return sensitive


#------------------------------------------------------------------------------------------
#finding out the descriptive data columns which may have sensitive data in the form of text
#------------------------------------------------------------------------------------------
def descriptive_columns(file_path):
    # Define keywords to filter out
    keywords = ["description", "remarks", "notes", "comments", "observations", "details", "summary", "explanation",
    "reviews", "feedback", "testimonials", "opinions", "assessment", "suggestions", "experience",
    "incident_report", "case_notes", "audit_notes", "findings", "status_update", "history", "progress_report",
    "additional_info", "clarifications", "justification", "annotations", "excerpts", "statement", "explanation_text"]

    # Ensure columns are properly loaded from CSV/Excel
    if file_path.endswith(".csv"):
        df = pd.read_csv(file_path, nrows=1)  # Read only header
    elif file_path.endswith((".xls", ".xlsx")):
        df = pd.read_excel(file_path, nrows=1, engine="openpyxl")  # Read only header
    else:
        raise ValueError("Unsupported file format. Please provide a CSV or Excel file.")

    # Get actual column names
    all_columns = df.columns.tolist()


    des=[col for col in all_columns if any(re.search(keyword, col, re.IGNORECASE) for keyword in keywords)]
    return des

#-----------------------------------------------------------------
#anonymizing descriptive values
#-----------------------------------------------------------------
def detect_noun_desc(text):
    doc = nlp(text)
    modified_text=[]
    for token in doc:
        if token.pos_ in ['PROPN']:
            modified_text.append("<sensitive>")
        else:
            modified_text.append(token.text)
    text=" ".join(modified_text)
    return text


#-------------------------------------------------------------------
#function to anonymize a excel file
#-------------------------------------------------------------------
def excel_an(input_file, output_file):
    df = pd.read_excel(input_file)  # Read as string for safety

    sensitive_old = detect_noun(input_file)
    desc = descriptive_columns(input_file)
    sensitive = list(set(sensitive_old) - set(desc))

    column_counters = {col: 1 for col in df.columns if col in sensitive_old}
    column_mappings = {col: {} for col in sensitive}  # Store mappings for each column
    mapping = {}

    # Anonymize sensitive columns while maintaining consistency
    for col in sensitive:
        new_values = []
        for val in df[col].astype(str):
            if pd.notna(val):
                if val in column_mappings[col]:
                    anonymized_value = column_mappings[col][val]  # Use existing mapping
                else:
                    anonymized_value = f"{col}{column_counters[col]}"
                    column_mappings[col][val] = anonymized_value  # Store new mapping
                    column_counters[col] += 1  # Increment counter

                mapping[anonymized_value] = val
                new_values.append(anonymized_value)
            else:
                new_values.append(val)

        df[col] = new_values

    # Anonymize descriptive columns
    for col in desc:
        for idx, val in enumerate(df[col].astype(str)):
            if pd.notna(val):
                an_values = detect_noun_desc(val)
                df.at[idx, col] = an_values
                mapping[an_values] = val

    df.to_excel(output_file, index=False, sheet_name="Anonymized Data")
    print(f"✅ Anonymized file saved as {output_file}")

    # Save mapping as JSON
    with open("mappings.json", "w") as f:
        json.dump(mapping, f)

#------------------------------------------------------
#function for de-anonymizing excel data
#------------------------------------------------------

def excel_dean(input_file, output_file, mapping_file):
    print("🔄 Loading data...")

    with open(mapping_file, "r") as f:
        mapping = json.load(f)

    df = pd.read_excel(input_file)  # Read as string for safety
    mapping_keys = set(mapping.keys())
    df = df.applymap(lambda x: mapping[x] if x in mapping_keys else x)
    df.to_excel(output_file, index=False, sheet_name="De-anonymized Data")

    print(f"✅ De-anonymized file saved as {output_file}")

#----------------------------------------------------------
#function for anonymizing csv data
#----------------------------------------------------------
def csv_an(input_file, output_file):
    df = pd.read_csv(input_file, engine="python")

    sensitive_old = detect_noun(input_file)
    desc = descriptive_columns(input_file)
    sensitive = list(set(sensitive_old) - set(desc))

    column_counters = {col: 1 for col in df.columns if col in sensitive_old}
    column_mappings = {col: {} for col in sensitive}  # Store mappings for each column

    # Anonymize sensitive columns while maintaining consistency
    for col in sensitive:
        new_values = []
        for val in df[col].astype(str):
            if pd.notna(val):
                if val in column_mappings[col]:
                    anonymized_value = column_mappings[col][val]  # Use existing mapping
                else:
                    anonymized_value = f"{col}{column_counters[col]}"
                    column_mappings[col][val] = anonymized_value  # Store new mapping
                    column_counters[col] += 1  # Increment counter

                mapping[anonymized_value] = val
                new_values.append(anonymized_value)
            else:
                new_values.append(val)

        df[col] = new_values

    # Anonymize descriptive columns
    for col in desc:
        for idx, val in enumerate(df[col].astype(str)):
            if pd.notna(val):
                an_values = detect_noun_desc(val)
                df.at[idx, col] = an_values
                mapping[an_values] = val

    df.to_csv(output_file, index=False)
    print(f"✅ Anonymized file saved as {output_file}")

    # Save mapping as JSON
    with open("mappings.json", "w") as f:
        json.dump(mapping, f)

#------------------------------------------------------------
#function for de anonymizing csv data
#------------------------------------------------------------
def csv_dean(input_file, output_file, mapping_file):
    print("🔄 Loading data...")

    with open(mapping_file, "r") as f:
        mapping = json.load(f)
    df = pd.read_csv(input_file, engine="python", dtype=str)  # Read as string for safety
    mapping_keys = set(mapping.keys())
    df = df.applymap(lambda x: mapping[x] if x in mapping_keys else x)
    df.to_csv(output_file, index=False)

    print(f"✅ De-anonymized file saved as {output_file}")


#--------------------------------------------------------------
#function to determine whcih type of data is need to perform
#--------------------------------------------------------------
@time_it
def anonymization(input_file):
    if input_file.endswith(".csv"):
        csv_an(input_file,"intermediate.csv")
        csv_dean("intermediate.csv","deanonymized.csv","mappings.json")
    elif input_file.endswith(".xlsx"):
        excel_an(input_file,"intermediate.xlsx")
        excel_dean("intermediate.xlsx","deanonymized.xlsx","mappings.json")

uploaded = files.upload()
file_name = list(uploaded.keys())[0].strip().replace(' ', '-')
anonymization(file_name)

Saving testing-csv-ff.csv to testing-csv-ff.csv
✅ Anonymized file saved as intermediate.csv
🔄 Loading data...
✅ De-anonymized file saved as deanonymized.csv


Execution time anonymization: 0.533583 seconds


<ipython-input-8-c92c682ea52b>:220: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: mapping[x] if x in mapping_keys else x)


In [ ]:
# Testing simplified consolidated code
import re
import json
import time
import spacy
import pandas as pd
from google.colab import files
from presidio_analyzer import AnalyzerEngine
from presidio_anonymizer import AnonymizerEngine

mapping = {}
nlp = spacy.load("en_core_web_sm")

analyzer = AnalyzerEngine()
anonymizer = AnonymizerEngine()

def time_it(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f'\n\nExecution time {func.__name__}: {end-start:.6f} seconds')
        return result
    return wrapper

def read_file(file_path):
    """Read CSV or Excel file and return DataFrame."""
    if file_path.endswith(".csv"):
        return pd.read_csv(file_path, engine="python"), "csv"
    elif file_path.endswith(".xlsx"):
        return pd.read_excel(file_path, engine="openpyxl"), "excel"
    else:
        raise ValueError("Unsupported file format. Please provide a CSV or Excel file.")

def detect_noun(df):
    sensitive = []

    for col in df.columns:
        if df[col].dtype in ['int64', 'float64'] or re.search(r'\b(id|rate|amount|price|cost|value|score|percentage)\b', col, re.IGNORECASE):
            continue  # Skip numeric and financial data

        text_samples = df[col].astype(str).head(10)
        noun_count, total_count = 0, 0

        for value in text_samples:
            if value.replace(".", "").isdigit() or '%' in value:
                continue

            doc = nlp(value)
            for token in doc:
                if token.pos_ in ['PROPN', 'NOUN']:
                    noun_count += 1
                total_count += 1

        if total_count > 0 and (noun_count / total_count) > 0.5:
            sensitive.append(col)

    print('\n\nSensitive cols:', sensitive)
    return sensitive

def detect_id_columns(df):
    return [col for col in df.columns if re.search(r'\b(id)$\b', col, re.IGNORECASE)]

def descriptive_columns(df):
    keywords = ["description", "remarks", "notes", "comments", "observations", "details", "summary", "explanation", "reviews", "feedback", "history"]
    return [col for col in df.columns if any(re.search(keyword, col, re.IGNORECASE) for keyword in keywords)]

def detect_pii(text):
    results = analyzer.analyze(text=text, entities=["PHONE_NUMBER", "CREDIT_CARD"], language="en")
    return anonymizer.anonymize(text=text, analyzer_results=results).text if results else text

def detect_pii_columns(df):
    """Detect columns that contain PII data (phone, credit card, ID)."""
    pii_columns = []

    for col in df.columns:
        sample_texts = df[col].astype(str).head(10).dropna().tolist()
        for text in sample_texts:
            if analyzer.analyze(text=text, entities=["PHONE_NUMBER", "CREDIT_CARD"], language="en"):
                pii_columns.append(col)
                break

    print("\n\nPII Columns:", pii_columns)
    return pii_columns

def anonymize_data(df):
    sensitive = detect_noun(df)
    id_cols = detect_id_columns(df)
    desc = descriptive_columns(df)
    pii_cols = detect_pii_columns(df)
    all_sensitive_cols = set(sensitive + id_cols + pii_cols)  # Combine all detected sensitive columns

    column_counters = {col: 1 for col in all_sensitive_cols}
    column_mappings = {col: {} for col in all_sensitive_cols}

    for col in all_sensitive_cols:
        new_values = []
        for val in df[col].astype(str):
            if pd.notna(val):
                if val in column_mappings[col]:
                    anonymized_value = column_mappings[col][val]
                else:
                    anonymized_value = f"{col}{column_counters[col]}"
                    column_mappings[col][val] = anonymized_value
                    column_counters[col] += 1
                mapping[anonymized_value] = val
                new_values.append(anonymized_value)
            else:
                new_values.append(val)
        df[col] = new_values

    for col in desc:
        df[col] = df[col].astype(str).apply(detect_pii)  # PII masking for descriptive columns

    return df

def save_file(df, file_type, output_file):
    if file_type == "csv":
        df.to_csv(output_file, index=False)
    else:
        df.to_excel(output_file, index=False, sheet_name="Anonymized Data")

@time_it
def anonymization(input_file):
    df, file_type = read_file(input_file)
    df = anonymize_data(df)
    output_file = f"anonymized.{file_type}"
    save_file(df, file_type, output_file)
    with open("mappings.json", "w") as f:
        json.dump(mapping, f)
    print(f"✅ Anonymized file saved as {output_file}")

@time_it
def de_anonymization(input_file, mapping_file):
    df, file_type = read_file(input_file)
    with open(mapping_file, "r") as f:
        mapping = json.load(f)
    df = df.applymap(lambda x: mapping.get(x, x))
    output_file = f"deanonymized.{file_type}"
    save_file(df, file_type, output_file)
    print(f"✅ De-anonymized file saved as {output_file}")

# uploaded = files.upload()
# file_name = list(uploaded.keys())[0].strip().replace(' ', '-')
# anonymization(file_name)
anonymization(input_file = 'testing-csv-ff.csv')
de_anonymization(file_name, "mappings.json")




Sensitive cols: ['Reporting Month', 'Cluster Name', 'Input Account Name', 'Account Name']


PII Columns: ['Mobile Number', 'Credit Card Number']
✅ Anonymized file saved as anonymized.csv


Execution time anonymization: 4.649121 seconds


<ipython-input-38-7c67c9f901aa>:138: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: mapping.get(x, x))


✅ De-anonymized file saved as deanonymized.csv


Execution time de_anonymization: 0.777546 seconds


In [ ]:
# Masking prompt from values in mappings.json
import json

def mask_prompt(prompt, mapping_file="mappings.json"):
    """Replaces original values in the prompt with masked values from mappings.json."""
    with open(mapping_file, "r") as f:
        mapping = json.load(f)

    reversed_mapping = {v: k for k, v in mapping.items()}  # Reverse mapping for lookup
    for original, masked in reversed_mapping.items():
        if original in prompt:
            prompt = prompt.replace(original, masked)

    return prompt

def unmask_response(response, mapping_file="mappings.json"):
    """Replaces masked values in the response with original values from mappings.json."""
    with open(mapping_file, "r") as f:
        mapping = json.load(f)

    for masked, original in mapping.items():
        if masked in response:
            response = response.replace(masked, original)

    return response

prompt_1 = "Please analyze the sales performance for Gamma Inc in Mar-24. The key account manager is Alex Brown, and their contact number is 7654321098. Employee ID 35678 was responsible for managing this account. Identify any trends or anomalies."
prompt_2 = "For the North cluster, examine the transactions linked to Liam Johnson in Apr-24. His registered mobile number is 5432109876, and the associated credit card number is 4539876543210987. Provide insights on spending patterns and any unusual activities."
prompt_3 = "Check the financial records for Theta Corp in Jul-24. The primary contact is William Young, and his phone number is 7654321096. Employee ID 55667 handled this account. Assess if there were any fraudulent transactions."

response_1 = mask_prompt(prompt_1)

print(response_1)
print(unmask_response(response_1))

Please analyze the sales performance for Input Account Name3 in Reporting Month3. The key account manager is Account Name3, and their contact number is Mobile Number3. Employee ID Employee ID3 was responsible for managing this account. Identify any trends or anomalies.
Please analyze the sales performance for Gamma Inc in Mar-24. The key account manager is Alex Brown, and their contact number is 7654321098. Employee ID 35678 was responsible for managing this account. Identify any trends or anomalies.


In [ ]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 18.6 MB/s eta 0:00:00


In [ ]:
# Implementing using Faker
import re
import json
import time
import spacy
import pandas as pd
from faker import Faker
from google.colab import files
from presidio_analyzer import AnalyzerEngine
from presidio_anonymizer import AnonymizerEngine

# Initialize dependencies
fake = Faker()
nlp = spacy.load("en_core_web_sm")

analyzer = AnalyzerEngine()
anonymizer = AnonymizerEngine()

mapping = {}  # Stores original-to-anonymized values for de-anonymization

def time_it(func):
    """Decorator to measure execution time."""
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f'\n\nExecution time {func.__name__}: {end-start:.6f} seconds')
        return result
    return wrapper

def read_file(file_path):
    """Read CSV or Excel file and return DataFrame."""
    if file_path.endswith(".csv"):
        return pd.read_csv(file_path, engine="python"), "csv"
    elif file_path.endswith(".xlsx"):
        return pd.read_excel(file_path, engine="openpyxl"), "excel"
    else:
        raise ValueError("Unsupported file format. Please provide a CSV or Excel file.")

def detect_noun(df):
    """Detect columns that contain mostly noun-based text data."""
    sensitive = []

    for col in df.columns:
        if df[col].dtype in ['int64', 'float64'] or re.search(r'\b(id|rate|amount|price|cost|value|score|percentage)\b', col, re.IGNORECASE):
            continue  # Skip numeric and financial data

        text_samples = df[col].astype(str).head(10)
        noun_count, total_count = 0, 0

        for value in text_samples:
            if value.replace(".", "").isdigit() or '%' in value:
                continue

            doc = nlp(value)
            for token in doc:
                if token.pos_ in ['PROPN', 'NOUN']:
                    noun_count += 1
                total_count += 1

        if total_count > 0 and (noun_count / total_count) > 0.5:
            sensitive.append(col)

    print('\n\nSensitive cols:', sensitive)
    return sensitive

def detect_id_columns(df):
    """Detect columns that contain ID values."""
    return [col for col in df.columns if re.search(r'\b(id)$\b', col, re.IGNORECASE)]

def descriptive_columns(df):
    """Detect columns that contain descriptive or free-text data."""
    keywords = ["description", "remarks", "notes", "comments", "observations", "details", "summary", "explanation", "reviews", "feedback", "history"]
    return [col for col in df.columns if any(re.search(keyword, col, re.IGNORECASE) for keyword in keywords)]

def detect_pii(text):
    """Detect and anonymize PII data in text fields."""
    results = analyzer.analyze(text=text, entities=["PHONE_NUMBER", "CREDIT_CARD"], language="en")
    return anonymizer.anonymize(text=text, analyzer_results=results).text if results else text

def detect_pii_columns(df):
    """Detect columns that contain PII data (phone, credit card, etc.)."""
    pii_columns = []

    for col in df.columns:
        sample_texts = df[col].astype(str).head(10).dropna().tolist()
        for text in sample_texts:
            if analyzer.analyze(text=text, entities=["PHONE_NUMBER", "CREDIT_CARD"], language="en"):
                pii_columns.append(col)
                break

    print("\n\nPII Columns:", pii_columns)
    return pii_columns

# Mapping Faker functions to relevant column types
faker_mapping = {
    "name": fake.name,
    "email": fake.email,
    "phone": fake.phone_number,
    "address": fake.address,
    "uuid": fake.uuid4,
    "credit_card": fake.credit_card_number,
}

def get_faker_value(column_name):
    """Return a Faker-generated value based on column type."""
    if re.search(r'\b(name)\b', column_name, re.IGNORECASE):
        return faker_mapping["name"]()
    if re.search(r'\b(email)\b', column_name, re.IGNORECASE):
        return faker_mapping["email"]()
    if re.search(r'\b(phone|mobile|contact)\b', column_name, re.IGNORECASE):
        return faker_mapping["phone"]()
    if re.search(r'\b(address|location)\b', column_name, re.IGNORECASE):
        return faker_mapping["address"]()
    if re.search(r'\b(id)\b', column_name, re.IGNORECASE):
        return faker_mapping["uuid"]()
    if re.search(r'\b(credit_card|card_number|credit card)\b', column_name, re.IGNORECASE):
        return faker_mapping["credit_card"]()
    return fake.word()  # Default fallback

def anonymize_data(df):
    """Anonymize sensitive columns using Faker."""
    sensitive = detect_noun(df)
    id_cols = detect_id_columns(df)
    desc = descriptive_columns(df)
    pii_cols = detect_pii_columns(df)
    all_sensitive_cols = set(sensitive + id_cols + pii_cols)

    column_mappings = {col: {} for col in all_sensitive_cols}

    for col in all_sensitive_cols:
        new_values = []
        for val in df[col].astype(str):
            if pd.notna(val):
                if val in column_mappings[col]:
                    anonymized_value = column_mappings[col][val]
                else:
                    anonymized_value = get_faker_value(col)
                    column_mappings[col][val] = anonymized_value
                mapping[anonymized_value] = val
                new_values.append(anonymized_value)
            else:
                new_values.append(val)
        df[col] = new_values

    for col in desc:
        df[col] = df[col].astype(str).apply(detect_pii)  # Mask PII in descriptive text

    return df

def save_file(df, file_type, output_file):
    """Save the DataFrame to a file."""
    if file_type == "csv":
        df.to_csv(output_file, index=False)
    else:
        df.to_excel(output_file, index=False, sheet_name="Anonymized Data")

@time_it
def anonymization(input_file):
    """Perform anonymization on a file."""
    df, file_type = read_file(input_file)
    df = anonymize_data(df)
    output_file = f"anonymized.{file_type}"
    save_file(df, file_type, output_file)

    # Save mapping for de-anonymization
    with open("mappings.json", "w") as f:
        json.dump(mapping, f)

    print(f"✅ Anonymized file saved as {output_file}")

@time_it
def de_anonymization(input_file, mapping_file):
    """Revert anonymized data to its original values using mappings."""
    df, file_type = read_file(input_file)

    with open(mapping_file, "r") as f:
        mapping = json.load(f)

    df = df.applymap(lambda x: mapping.get(x, x))
    output_file = f"deanonymized.{file_type}"
    save_file(df, file_type, output_file)

    print(f"✅ De-anonymized file saved as {output_file}")

uploaded = files.upload()
file_name = list(uploaded.keys())[0].strip().replace(' ', '-')
anonymization(file_name)
# anonymization("testing-csv-ff.csv")
de_anonymization("anonymized.csv", "mappings.json")


Saving testing-csv-ff.csv to testing-csv-ff.csv


Sensitive cols: ['Reporting Month', 'Cluster Name', 'Input Account Name', 'Account Name']


PII Columns: ['Mobile Number', 'Credit Card Number']
✅ Anonymized file saved as anonymized.csv


Execution time anonymization: 2.413634 seconds
✅ De-anonymized file saved as deanonymized.csv


Execution time de_anonymization: 0.007664 seconds


<ipython-input-3-322bf1fc318e>:180: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: mapping.get(x, x))


# **TESTING FAKER CODE**

In [ ]:
!pip install flair
!pip install faker
!pip install presidio_analyzer
!pip install presidio_anonymizer
# !python -m spacy download en_core_web_lg
!pip install --upgrade --force-reinstall "numpy==1.26.4"
!pip install --upgrade --force-reinstall "Cython" "spacy"

In [ ]:
# Testing Faker code (1)
import pandas as pd
from presidio_analyzer import AnalyzerEngine
from faker import Faker

def time_it(func):
    """Decorator to measure execution time."""
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f'\n\nExecution time {func.__name__}: {end-start:.6f} seconds')
        return result
    return wrapper

@time_it
def analyze_column(df):
  entity_columns = {}  # Initialize as a dictionary
  for col in df.columns:
    unique_values = df[col].dropna().astype(str).unique()[:25]
    entity_counts = {}

    for value in unique_values:
      results = analyzer.analyze(text=value, language="en")
      for result in results:
        entity_counts[result.entity_type] = entity_counts.get(result.entity_type, 0) + 1
    if entity_counts:
      predominant_entity = max(entity_counts, key=entity_counts.get)
      if predominant_entity not in entity_columns:
        entity_columns[predominant_entity] = []  # Assign a list to the entity key
      entity_columns[predominant_entity].append(col)
  return entity_columns  # Return the dictionary

def load_and_analyze(file_path):
  if file_path.endswith(".csv"):
    df = pd.read_csv(file_path, engine="python")
  elif file_path.endswith((".xls", ".xlsx")):
    df = pd.read_excel(file_path)
  else:
    raise ValueError("Unsupported file format. Provide a CSV or Excel file.")

  df_sample = df.drop_duplicates().head()
  classified_columns = analyze_column(df_sample)
  return classified_columns  # Return the dictionary

In [ ]:
# Testing Faker code (2)
file="smaller_companies.csv"
classified_columns=load_and_analyze(file)
df=pd.read_csv(file)
print(classified_columns)



Execution time analyze_column: 0.468736 seconds
{'DATE_TIME': ['id', 'size range'], 'LOCATION': ['name', 'locality', 'country'], 'URL': ['domain', 'linkedin url'], 'US_DRIVER_LICENSE': ['current employee estimate', 'total employee estimate']}


In [ ]:
# Testing Faker code (3)
import os
import json
from faker import Faker

fake = Faker()
Faker.seed(42)

# data_mapping.json
MAPPING_FILE = "output/mappings.json"
forward_mapping = {}
reverse_mapping = {}

@time_it
def load_mapping():
    """Loads existing mappings from a file if available."""
    global forward_mapping, reverse_mapping
    if os.path.exists(MAPPING_FILE):
        try:
            with open(MAPPING_FILE, "r") as f:
                data = json.load(f)
                # Convert keys back to tuples if needed
                forward_mapping = {tuple(eval(k)): v for k, v in data.get("forward_mapping", {}).items()}
                reverse_mapping = {tuple(eval(k)): v for k, v in data.get("reverse_mapping", {}).items()}
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON file: {e}")
            # If the file is invalid, reset the mappings
            forward_mapping = {}
            reverse_mapping = {}

def save_mapping():
    """Saves the mapping to a JSON file."""
    os.makedirs(os.path.dirname(MAPPING_FILE), exist_ok=True)
    # Convert tuple keys to strings before saving
    modified_forward_mapping = {str(k): v for k, v in forward_mapping.items()}
    modified_reverse_mapping = {str(k): v for k, v in reverse_mapping.items()}
    with open(MAPPING_FILE, "w") as f:
        json.dump({"forward_mapping": modified_forward_mapping, "reverse_mapping": modified_reverse_mapping}, f, indent=4)

@time_it
def generate_fake_value(original: str, entity_type: str) -> str:
    """Generates fake values while ensuring uniqueness."""
    if original in forward_mapping:
        return forward_mapping[original]

    faker_mapping = {
        "PERSON": fake.name,
        "FIRST_NAME": fake.first_name,
        "LAST_NAME": fake.last_name,
        "EMAIL": fake.email,
        "URL": fake.url,
        "PHONE_NUMBER": fake.phone_number,
        "CREDIT_CARD": fake.credit_card_number,
        "IBAN": fake.iban,
        "US_SSN": fake.ssn,
        # "DATE": fake.date_of_birth,
        "LOCATION": fake.company,
        "STREET_ADDRESS": fake.street_address,
        "CITY": fake.city,
        "STATE": fake.state,
        "COUNTRY": fake.country,
        "ZIP_CODE": fake.zipcode,
        "ORGANIZATION": fake.company,
        "JOB_TITLE": fake.job,
        "USERNAME": fake.user_name,
        "PASSWORD": fake.password,
        "IP_ADDRESS": fake.ipv4,
        "MAC_ADDRESS": fake.mac_address,
        "LICENSE_PLATE": fake.license_plate,
        "UUID": fake.uuid4,
        "BANK_ACCOUNT": fake.bban,
        "TRANSACTION_ID": fake.uuid4,
        "DEVICE_ID": fake.uuid4,
    }

    fake_value = faker_mapping.get(entity_type, lambda: original)()
    forward_mapping[original] = fake_value
    reverse_mapping[fake_value] = original
    return fake_value

@time_it
def mask_sensitive_data(df: pd.DataFrame, classified_columns: dict) -> pd.DataFrame:
    """Masks sensitive columns using Faker based on classified entity types."""
    for entity_type, columns in classified_columns.items():
        for column in columns:
            if column in df.columns:
                df[column] = df[column].astype(str).apply(lambda x: generate_fake_value(x, entity_type) if x.strip() else x)
    return df

# testing-csv-ff.csv
# smaller_companies.csv
if __name__ == "__main__":
    load_mapping()
    input_file = "smaller_companies.csv"
    output_file = "output/anonymized.csv"

    df = pd.read_csv(input_file)
    classified_columns = load_and_analyze("smaller_companies.csv")

    df = mask_sensitive_data(df, classified_columns)
    df.to_csv(output_file, index=False)
    save_mapping()

In [ ]:
# Hashing file so as to reuse it (FAKER)
import os
import json
import pandas as pd
import hashlib
from faker import Faker

fake = Faker()
Faker.seed(42)

MAPPING_FILE = "mappings.json"  # Store mappings directly
N_LINES_FOR_HASH = 100  # Number of lines to consider for hashing

# Load existing mappings if available
if os.path.exists(MAPPING_FILE):
    with open(MAPPING_FILE, "r") as f:
        try:
            mappings_data = json.load(f)
        except json.JSONDecodeError:
            mappings_data = {}
else:
    mappings_data = {}

def compute_file_hash(file_path, num_lines=N_LINES_FOR_HASH):
    """Compute a quick hash based on the first `num_lines` of the file."""
    hasher = hashlib.sha256()
    with open(file_path, "r", encoding="utf-8") as f:
        for _ in range(num_lines):
            line = f.readline()
            if not line:
                break
            hasher.update(line.encode("utf-8"))
    return hasher.hexdigest()

def save_mapping(filename, file_hash, forward_mapping, reverse_mapping):
    """Save mappings along with filename and file hash."""
    mappings_data[filename] = {
        "file_hash": file_hash,
        "forward_mapping": forward_mapping,
        "reverse_mapping": reverse_mapping
    }
    with open(MAPPING_FILE, "w") as f:
        json.dump(mappings_data, f, indent=4)

def load_mapping(filename, file_hash):
    """Load mapping if filename and file hash match."""
    if filename in mappings_data and mappings_data[filename]["file_hash"] == file_hash:
        return mappings_data[filename]["forward_mapping"], mappings_data[filename]["reverse_mapping"]
    return {}, {}

def generate_fake_value(original: str, entity_type: str, forward_mapping, reverse_mapping):
    """Generates fake values while ensuring uniqueness."""
    if original in forward_mapping:
        return forward_mapping[original]

    faker_mapping = {
        "PERSON": fake.name,
        "FIRST_NAME": fake.first_name,
        "LAST_NAME": fake.last_name,
        "EMAIL": fake.email,
        "URL": fake.url,
        "PHONE_NUMBER": fake.phone_number,
        "CREDIT_CARD": fake.credit_card_number,
        "IBAN": fake.iban,
        "US_SSN": fake.ssn,
        "LOCATION": fake.company,
        "STREET_ADDRESS": fake.street_address,
        "CITY": fake.city,
        "STATE": fake.state,
        "COUNTRY": fake.country,
        "ZIP_CODE": fake.zipcode,
        "ORGANIZATION": fake.company,
        "JOB_TITLE": fake.job,
        "USERNAME": fake.user_name,
        "PASSWORD": fake.password,
        "IP_ADDRESS": fake.ipv4,
        "MAC_ADDRESS": fake.mac_address,
        "LICENSE_PLATE": fake.license_plate,
        "UUID": fake.uuid4,
        "BANK_ACCOUNT": fake.bban,
        "TRANSACTION_ID": fake.uuid4,
        "DEVICE_ID": fake.uuid4,
    }

    fake_value = faker_mapping.get(entity_type, lambda: original)()
    forward_mapping[original] = fake_value
    reverse_mapping[fake_value] = original
    return fake_value

def mask_sensitive_data(df: pd.DataFrame, classified_columns: dict, forward_mapping, reverse_mapping):
    """Masks sensitive columns using Faker based on classified entity types."""
    for entity_type, columns in classified_columns.items():
        for column in columns:
            if column in df.columns:
                df[column] = df[column].astype(str).apply(lambda x: generate_fake_value(x, entity_type, forward_mapping, reverse_mapping) if x.strip() else x)
    return df

# Run masking process
if __name__ == "__main__":
    input_file = "smaller_companies.csv"
    output_file = "anonymized.csv"

    # Compute file hash
    file_hash = compute_file_hash(input_file)

    # Load or initialize mappings
    forward_mapping, reverse_mapping = load_mapping(input_file, file_hash)

    df = pd.read_csv(input_file)
    classified_columns = load_and_analyze(input_file)  # Ensure this function is defined

    df = mask_sensitive_data(df, classified_columns, forward_mapping, reverse_mapping)
    df.to_csv(output_file, index=False)

    # Save updated mapping
    save_mapping(input_file, file_hash, forward_mapping, reverse_mapping)


In [ ]:
# Tried out finding total unique names
from faker import Faker
from faker.exceptions import UniquenessException

fake = Faker()
fake.unique.clear()
unique_names = set()

try:
  while True:
    unique_names.add(fake.unique.name())
except UniquenessException:
  print(f'Unique names length:', len(unique_names))

In [ ]:
# Masking ID in column headers

import os
import re
import time
import json
import pandas as pd
from faker import Faker
from presidio_analyzer import AnalyzerEngine

# Presidio Analyzer
analyzer = AnalyzerEngine()

# Faker initalization
fake = Faker()
Faker.seed(42)

# mappings.json
MAPPING_FILE = "mappings.json"
forward_mapping = {}
reverse_mapping = {}

def time_it(func):
    """Decorator to measure execution time."""
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f'\n\nExecution time {func.__name__}: {end-start:.6f} seconds')
        return result
    return wrapper

@time_it
def analyze_column(df):
  entity_columns = {}  # Initialize as a dictionary
  for col in df.columns:
    unique_values = df[col].dropna().astype(str).unique()[:15]
    entity_counts = {}

    for value in unique_values:
      results = analyzer.analyze(text=value, language="en")
      for result in results:
        entity_counts[result.entity_type] = entity_counts.get(result.entity_type, 0) + 1
    if entity_counts:
      predominant_entity = max(entity_counts, key=entity_counts.get)
      if predominant_entity not in entity_columns:
        entity_columns[predominant_entity] = []  # Assign a list to the entity key
      entity_columns[predominant_entity].append(col)
  return entity_columns  # Return the dictionary

def load_and_analyze(file_path):
  if file_path.endswith(".csv"):
    df = pd.read_csv(file_path, engine="python")
  elif file_path.endswith((".xls", ".xlsx")):
    df = pd.read_excel(file_path)
  else:
    raise ValueError("Unsupported file format. Provide a CSV or Excel file.")

  df_sample = df.drop_duplicates().head()
  classified_columns = analyze_column(df_sample)
  return classified_columns

def load_mapping():
    """Loads existing mappings from a file if available."""
    global forward_mapping, reverse_mapping
    if os.path.exists(MAPPING_FILE):
        try:
            with open(MAPPING_FILE, "r") as f:
                data = json.load(f)
                forward_mapping = data.get("forward_mapping", {})
                reverse_mapping = data.get("reverse_mapping", {})
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON file: {e}")
            forward_mapping = {}
            reverse_mapping = {}

def save_mapping():
    """Saves the mapping to a JSON file."""
    with open(MAPPING_FILE, "w") as f:
        json.dump({"forward_mapping": forward_mapping, "reverse_mapping": reverse_mapping}, f, indent=4)

def generate_fake_value(original: str, entity_type: str) -> str:
    """Generates fake values while ensuring uniqueness."""
    if original in forward_mapping:
        return forward_mapping[original]

    faker_mapping = {
        "PERSON": fake.name,
        "FIRST_NAME": fake.first_name,
        "LAST_NAME": fake.last_name,
        "EMAIL": fake.email,
        "URL": fake.url,
        "PHONE_NUMBER": fake.phone_number,
        "CREDIT_CARD": fake.credit_card_number,
        "IBAN": fake.iban,
        "US_SSN": fake.ssn,
        "LOCATION": fake.company,
        "STREET_ADDRESS": fake.street_address,
        "CITY": fake.city,
        "STATE": fake.state,
        "COUNTRY": fake.country,
        "ZIP_CODE": fake.zipcode,
        "ORGANIZATION": fake.company,
        "JOB_TITLE": fake.job,
        "USERNAME": fake.user_name,
        "PASSWORD": fake.password,
        "IP_ADDRESS": fake.ipv4,
        "MAC_ADDRESS": fake.mac_address,
        "LICENSE_PLATE": fake.license_plate,
        "UUID": fake.uuid4,
        "BANK_ACCOUNT": fake.bban,
        "TRANSACTION_ID": fake.uuid4,
        "DEVICE_ID": fake.uuid4,
        "ID": fake.uuid4,  # Generating fake values for ID columns
    }

    fake_value = faker_mapping.get(entity_type, lambda: original)()
    forward_mapping[original] = fake_value
    reverse_mapping[fake_value] = original
    return fake_value

@time_it
def mask_sensitive_data(df: pd.DataFrame, classified_columns: dict) -> pd.DataFrame:
    """Masks sensitive columns using Faker based on classified entity types."""
    for entity_type, columns in classified_columns.items():
        for column in columns:
            if column in df.columns:
                df[column] = df[column].astype(str).apply(lambda x: generate_fake_value(x, entity_type) if x.strip() else x)

    # Find columns with 'id' in the name using regex and mask them
    id_columns = [col for col in df.columns if re.search(r'\bid\b', col, re.IGNORECASE)]
    print('ID cols:', id_columns)
    for col in id_columns:
        df[col] = df[col].astype(str).apply(lambda x: generate_fake_value(x, "ID") if x.strip() else x)

    return df


# smaller_companies.csv
# testing-csv-ff.csv
if __name__ == "__main__":
    load_mapping()
    input_file = "smaller_companies.csv"
    output_file = "anonymized.csv"

    df = pd.read_csv(input_file)
    classified_columns = load_and_analyze(input_file)

    df = mask_sensitive_data(df, classified_columns)
    df.to_csv(output_file, index=False)
    save_mapping()


In [ ]:
# Testing columnwise replacing using Faker
import os
import re
import time
import json
import pandas as pd
from faker import Faker
from presidio_analyzer import AnalyzerEngine

# Initialize Faker & Presidio Analyzer
fake = Faker()
Faker.seed(42)
analyzer = AnalyzerEngine()

# Mapping file for storing forward/reverse mappings
MAPPING_FILE = "mappings.json"
forward_mapping = {}
reverse_mapping = {}

def time_it(func):
    """Decorator to measure execution time."""
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f'\nExecution time {func.__name__}: {end-start:.6f} seconds')
        return result
    return wrapper

def load_mapping():
    """Loads existing mappings from a file if available."""
    global forward_mapping, reverse_mapping
    if os.path.exists(MAPPING_FILE):
        try:
            with open(MAPPING_FILE, "r") as f:
                data = json.load(f)
                forward_mapping = data.get("forward_mapping", {})
                reverse_mapping = data.get("reverse_mapping", {})
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON file: {e}")
            forward_mapping = {}
            reverse_mapping = {}

def save_mapping():
    """Saves the mapping to a JSON file."""
    with open(MAPPING_FILE, "w") as f:
        json.dump({"forward_mapping": forward_mapping, "reverse_mapping": reverse_mapping}, f, indent=4)

@time_it
def analyze_column(df):
    """Detects sensitive columns using Presidio."""
    entity_columns = {}
    for col in df.columns:
        unique_values = df[col].dropna().astype(str).unique()[:10]
        entity_counts = {}

        for value in unique_values:
            results = analyzer.analyze(text=value, language="en")
            for result in results:
                entity_counts[result.entity_type] = entity_counts.get(result.entity_type, 0) + 1

        if entity_counts:
            predominant_entity = max(entity_counts, key=entity_counts.get)
            if predominant_entity not in entity_columns:
                entity_columns[predominant_entity] = []
            entity_columns[predominant_entity].append(col)

    return entity_columns

def load_and_analyze(file_path):
    """Loads CSV/Excel and analyzes sensitive columns."""
    df = pd.read_csv(file_path) if file_path.endswith(".csv") else pd.read_excel(file_path)
    return analyze_column(df)

def generate_fake_values(df, entity_type, column):
    """Generates fake values for an entire column while preserving uniqueness."""
    faker_mapping = {
        "PERSON": fake.name,
        "FIRST_NAME": fake.first_name,
        "LAST_NAME": fake.last_name,
        "EMAIL": fake.email,
        "URL": fake.url,
        "PHONE_NUMBER": fake.phone_number,
        "CREDIT_CARD": lambda: fake.credit_card_number(card_type=None),
        "IBAN": fake.iban,
        "US_SSN": fake.ssn,
        "LOCATION": fake.company,
        "STREET_ADDRESS": fake.street_address,
        "CITY": fake.city,
        "STATE": fake.state,
        "COUNTRY": fake.country,
        "ZIP_CODE": fake.zipcode,
        "ORGANIZATION": fake.company,
        "JOB_TITLE": fake.job,
        "USERNAME": fake.user_name,
        "PASSWORD": fake.password,
        "IP_ADDRESS": fake.ipv4,
        "MAC_ADDRESS": fake.mac_address,
        "LICENSE_PLATE": fake.license_plate,
        "UUID": fake.uuid4,
        "BANK_ACCOUNT": fake.bban,
        "TRANSACTION_ID": fake.uuid4,
        "DEVICE_ID": fake.uuid4,
        "ID": fake.uuid4,
        "US_BANK_NUMBER": lambda: fake.credit_card_number(card_type="visa")
    }

    if entity_type not in faker_mapping:
        return df  # Skip if no Faker function exists

    faker_func = faker_mapping[entity_type]

    # Convert numeric columns to strings before applying Faker mapping
    if df[column].dtype in [int, float]:
        df[column] = df[column].astype(str)

    original_values = df[column].dropna().astype(str).unique()
    fake_values = [forward_mapping.get(value, faker_func()) for value in original_values]

    # Store mappings
    for original, fake_val in zip(original_values, fake_values):
        forward_mapping[original] = str(fake_val)  # Ensure JSON serializable
        reverse_mapping[str(fake_val)] = original

    df[column] = df[column].map(forward_mapping).fillna(df[column])

    return df



def mask_id_columns(df):
    """Finds columns with 'id' in the name and replaces them with fake UUIDs."""
    id_columns = [col for col in df.columns if re.search(r'\bid\b', col, re.IGNORECASE)]

    for column in id_columns:
        unique_ids = df[column].astype(str).dropna().unique()
        fake_ids = [forward_mapping.get(value, str(fake.uuid4())) for value in unique_ids]

        # Store mappings for ID replacements
        for original, fake_id in zip(unique_ids, fake_ids):
            forward_mapping[original] = fake_id
            reverse_mapping[fake_id] = original

        # 🔹 Forcefully update df[column] with mapped values
        df[column] = df[column].astype(str).apply(lambda x: forward_mapping.get(x, x))

    return df


@time_it
def mask_sensitive_data(df, classified_columns):
    """Masks all detected sensitive columns using Faker."""
    for entity_type, columns in classified_columns.items():
        for column in columns:
            if column in df.columns:
                df = generate_fake_values(df, entity_type, column)

    # Mask all ID columns separately
    df = mask_id_columns(df)

    return df

# testing-csv-ff.csv
# smaller_companies.csv
if __name__ == "__main__":
    load_mapping()
    input_file = "smaller_100k_companies.csv"
    output_file = "anonymized.csv"

    df = pd.read_csv(input_file)
    classified_columns = load_and_analyze(input_file)

    df = mask_sensitive_data(df, classified_columns)
    df.to_csv(output_file, index=False)

    save_mapping()



Execution time analyze_column: 6.428666 seconds

Execution time mask_sensitive_data: 67.272321 seconds


In [ ]:
# Thejus code fast!
output_folder = "output_files"
class MaskingModule:
    def __init__(self, mapping_file: str = f"{output_folder}/data_mapping.json", seed: int = 42):
        self.fake = Faker()
        Faker.seed(seed)
        self.mapping_file = mapping_file
        self.forward_mapping = {}
        self.reverse_mapping = {}
        self._load_mapping_if_exists()
        print("✅ MaskingModule initialized.")

    def _generate_fake_column(self, column: pd.Series, column_name: str) -> pd.Series:
        """Generate fake data for a whole column at once and update mappings."""
        column_lower = column_name.lower()
        original_values = column.dropna().unique()
        fake_values = []

        if (
            "employee" in column_lower
            or "company" in column_lower
            or "account" in column_lower
        ):
            fake_values = [
                f"{self.fake.first_name()}{self.fake.random_number(digits=12)}{self.fake.first_name()}"
                for _ in original_values
            ]
        elif "project" in column_lower:
            fake_values = [
                f"PRJ-{self.fake.random_number(digits=12)}" for _ in original_values
            ]
        elif "entity" in column_lower:
            fake_values = [
                f"ENT-{self.fake.random_number(digits=12)}" for _ in original_values
            ]
        elif "program" in column_lower:
            fake_values = [
                f"PROG-{self.fake.random_number(digits=12)}" for _ in original_values
            ]
        else:
            fake_values = [
                f"{self.fake.first_name()}-{self.fake.random_number(digits=20)}-{self.fake.first_name()}"
                for _ in original_values
            ]

        mapping = dict(zip(original_values, fake_values))
        self.forward_mapping.update(mapping)
        self.reverse_mapping.update({v: k for k, v in mapping.items()})

        return column.map(mapping).fillna(column)

    def _save_mapping(self):
        """Save mappings to JSON with string keys and serializable values."""

        def convert_to_serializable(obj):
            if isinstance(
                obj, (np.integer, np.int64, np.int32)
            ):  # Convert numpy int to Python int
                return int(obj)
            elif isinstance(
                obj, (np.floating, np.float64, np.float32)
            ):  # Convert numpy float to Python float
                return float(obj)
            elif isinstance(obj, np.ndarray):  # Convert numpy array to list
                return obj.tolist()
            else:
                return obj  # Return as-is if already serializable

        mapping_data = {
            "forward_mapping": {
                str(k): convert_to_serializable(v) for k, v in self.forward_mapping.items()
            },
            "reverse_mapping": {
                str(k): convert_to_serializable(v) for k, v in self.reverse_mapping.items()
            },
            "metadata": {
                "updated_at": datetime.now().isoformat(),
                "record_count": len(self.forward_mapping),
            },
        }

        with open(self.mapping_file, "w") as f:
            json.dump(mapping_data, f, indent=4)

        print("💾 Mapping saved successfully.")

    def _load_mapping_if_exists(self):
        """Load existing mappings from JSON."""
        if os.path.exists(self.mapping_file):
            with open(self.mapping_file, "r") as f:
                data = json.load(f)
                self.forward_mapping = data["forward_mapping"]
                self.reverse_mapping = data["reverse_mapping"]
            print("📂 Existing mapping loaded.")

    def process_file(
        self,
        input_file: str,
        output_file: str,
        mode: str = "anonymize",
        include_columns: List[str] = None,
    ):
        """Process CSV and Excel files for anonymization and de-anonymization."""
        if mode not in ["anonymize", "deanonymize"]:
            raise ValueError("Mode must be either 'anonymize' or 'deanonymize'")

        file_ext = os.path.splitext(input_file)[-1].lower()
        df = (
            pd.read_csv(input_file) if file_ext == ".csv" else pd.read_excel(input_file)
        )
        print("📊 Data loaded successfully.")

        include_set = {"id", "name", "domain", "year", "founded", "industry", "size"}
        if include_columns:
            include_set.update(include_columns)

        for column in df.columns:
            if column in include_set:
                if mode == "anonymize":
                    df[column] = self._generate_fake_column(df[column], column)
                else:
                    df[column] = (
                        df[column]
                        .astype(str)
                        .map(self.reverse_mapping)
                        .where(df[column].notna(), df[column])
                    )

        df.to_csv(output_file, index=False) if file_ext == ".csv" else df.to_excel(
            output_file, index=False
        )
        if mode == "anonymize":
            self._save_mapping()
        print(f"✅ Data {mode}d and saved to {output_file}")

    def check_if_two_files_are_same(self, input_file: str, restored_file: str):
        """Check if the original and restored files match."""
        file_ext = os.path.splitext(input_file)[-1].lower()
        original_df = (
            pd.read_csv(input_file) if file_ext == ".csv" else pd.read_excel(input_file)
        )
        restored_df = (
            pd.read_csv(restored_file)
            if file_ext == ".csv"
            else pd.read_excel(restored_file)
        )

        test = original_df.equals(restored_df)
        print(f"📊 Are files identical? {test}")
        if not test:
            print("⚠️ Files are not identical!")
            print("🔍 Sample from original:")
            print(original_df.head(2))
            print("🔍 Sample from restored:")
            print(restored_df.head(2))
        else:
            print("✅ Files match perfectly!")

    def print_mapping_sample(self, n: int = 5):
        """Print sample mappings."""
        print("📌 Forward Mapping (Original → Anonymized):")
        for k, v in list(self.forward_mapping.items())[:n]:
            print(f"{k} → {v}")
        print("📌 Reverse Mapping (Anonymized → Original):")
        for k, v in list(self.reverse_mapping.items())[:n]:
            print(f"{k} → {v}")

    def mask_text(self, text, replace_dict):
        """Mask text using forward mapping."""
        for key, value in replace_dict.items():
            text = text.replace(key, value)
        return text



In [ ]:
# Optimizing Thejus's logic
import os
import re
import json
import time
import pandas as pd
import numpy as np
from faker import Faker
from datetime import datetime
from google.colab import files
from presidio_analyzer import AnalyzerEngine

# Initialize Faker & Presidio Analyzer
fake = Faker()
Faker.seed(42)
analyzer = AnalyzerEngine()

# Mapping file for storing forward/reverse mappings
MAPPING_FILE = "mappings.json"
forward_mapping = {}
reverse_mapping = {}

NUMERIC_PATTERN = re.compile(r"^[<>]?[\d,.%]+$")
SENSITIVE_HEADERS = {"phone", "mobile", "credit", "card", "id"}
EXCLUDED_ENTITIES = {"DATE_TIME"}  # Exclude unwanted entities


def time_it(func):
    """Decorator to measure execution time of functions."""
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f'\n⏳ Execution time {func.__name__}: {end-start:.6f} seconds')
        return result
    return wrapper


def load_mapping():
    """Loads existing mapping from file, ensuring proper data handling."""
    global forward_mapping, reverse_mapping
    if os.path.exists(MAPPING_FILE):
        try:
            with open(MAPPING_FILE, "r") as f:
                data = json.load(f)
                forward_mapping = data.get("forward_mapping", {})
                reverse_mapping = data.get("reverse_mapping", {})
            print("📂 Existing mapping loaded.")
        except json.JSONDecodeError as e:
            print(f"⚠️ Error decoding JSON file: {e}")


def save_mapping():
    """Saves the mapping to a JSON file with string keys."""
    mapping_data = {
        "forward_mapping": {str(k): str(v) for k, v in forward_mapping.items()},
        "reverse_mapping": {str(k): str(v) for k, v in reverse_mapping.items()},
        "metadata": {
            "updated_at": datetime.now().isoformat(),
            "record_count": len(forward_mapping),
        },
    }
    with open(MAPPING_FILE, "w") as f:
        json.dump(mapping_data, f, indent=4)
    print("💾 Mapping saved successfully.")



@time_it
def analyze_column(df):
    """Uses Presidio to classify columns based on detected entity types."""
    entity_columns = {}
    for col in df.columns:
        col_lower = col.lower()
        if any(keyword in col_lower for keyword in SENSITIVE_HEADERS):
            entity_columns.setdefault("SENSITIVE", []).append(col)
            continue

        unique_values = df[col].dropna().astype(str).unique()[:10]
        entity_counts = {}

        for value in unique_values:
            if NUMERIC_PATTERN.match(value):
                continue  # Skip purely numeric values unless it's a phone/credit card

            results = analyzer.analyze(text=value, language="en")
            for result in results:
                if result.entity_type in EXCLUDED_ENTITIES:
                    continue  # Skip unwanted entities
                entity_counts[result.entity_type] = entity_counts.get(result.entity_type, 0) + 1

        if entity_counts:
            predominant_entity = max(entity_counts, key=entity_counts.get)
            if predominant_entity not in entity_columns:
                entity_columns[predominant_entity] = []
            entity_columns[predominant_entity].append(col)

    return entity_columns


def generate_fake_values(column):
    """Generates consistent fake values for a column while preserving uniqueness."""
    col_name = column.name.lower()
    original_values = column.dropna().unique()

    if col_name in forward_mapping:
        mapping = forward_mapping[col_name]
    else:
        mapping = {}

    fake_values = []
    if re.search(r"\b(phone|mobile)\b", col_name, re.IGNORECASE):
        fake_values = [fake.phone_number() for _ in original_values]
    elif re.search(r"\b(credit|card)\b", col_name, re.IGNORECASE):
        fake_values = [fake.credit_card_number() for _ in original_values]
    elif re.search(r"\b(id|identifier)\b", col_name, re.IGNORECASE):
        fake_values = [fake.bothify(text="ID##########????") for _ in original_values]
    elif re.search(r"\b(name|full[_\s]?name|first[_\s]?name|last[_\s]?name)\b", col_name, re.IGNORECASE):
        fake_values = [f'{fake.name()--{fake.bothify(text="??????????")}}' for _ in original_values]
    else:
        fake_values = [fake.bothify(text="????########") for _ in original_values]


    mapping.update(dict(zip(original_values, fake_values)))
    forward_mapping[col_name] = mapping
    reverse_mapping[col_name] = {v: k for k, v in mapping.items()}

    return column.map(mapping).fillna(column)


@time_it
def mask_columnwise(df):
    """Replaces sensitive data only in columns detected as sensitive by Presidio."""
    sensitive_columns = analyze_column(df)

    for entity, cols in sensitive_columns.items():
        for column in cols:
            print(f"🔒 Masking sensitive column: {column} (Detected as {entity})")
            df[column] = generate_fake_values(df[column])

    return df


def restore_data(df):
    """Restores original values using stored mappings."""
    for column in df.columns:
        col_name = column.lower()
        if col_name in reverse_mapping:
            df[column] = df[column].astype(str).map(reverse_mapping[col_name]).fillna(df[column])
    return df


def check_if_files_match(original_file, restored_file):
    """Checks if the original and restored files match."""
    file_ext = os.path.splitext(original_file)[-1].lower()
    original_df = pd.read_csv(original_file) if file_ext == ".csv" else pd.read_excel(original_file)
    restored_df = pd.read_csv(restored_file) if file_ext == ".csv" else pd.read_excel(restored_file)
    test = original_df.equals(restored_df)
    print(f"📊 Are files identical? {test}")
    if not test:
        print("⚠️ Files are not identical!")
    else:
        print("✅ Files match perfectly!")


if __name__ == "__main__":
    load_mapping()
    uploaded = files.upload()
    input_file = list(uploaded.keys())[0].strip().replace(' ', '-')
    # input_file = "smaller_companies.csv" if not input_file else input_file

    anonymized_file = "anonymized.csv"
    restored_file = "restored.csv"

    df = pd.read_csv(input_file)
    df = mask_columnwise(df)  # Only masks detected sensitive columns
    df.to_csv(anonymized_file, index=False)
    save_mapping()

    df = restore_data(pd.read_csv(anonymized_file))
    df.to_csv(restored_file, index=False)

    check_if_files_match(input_file, restored_file)


In [ ]:
# V2
import os
import re
import json
import time
import pickle
import numpy as np
import pandas as pd
from faker import Faker
from datetime import datetime
from google.colab import files
from presidio_analyzer import AnalyzerEngine
# Initialize Faker & Presidio Analyzer
fake = Faker()
Faker.seed(42)
analyzer = AnalyzerEngine()

# Mapping file for storing forward/reverse mappings
MAPPING_FILE = "mappings.pkl"
forward_mapping = {}
reverse_mapping = {}

NUMERIC_PATTERN = re.compile(r"^[<>]?[\d,.%]+$")
SENSITIVE_HEADERS = {"phone", "mobile", "credit", "card", "id", "name"}
EXCLUDED_ENTITIES = {"DATE_TIME"}

def time_it(func):
    """Decorator to measure execution time of functions."""
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f'\n⏳ Execution time {func.__name__}: {end-start:.6f} seconds')
        return result
    return wrapper

def de_anonymize_paragraph(text):
  for category,mapping in reverse_mapping.items():
    for fake_value,original_value in mapping.items():
      if fake_value in text:
        text=text.replace(fake_value,original_value)
  return text

def load_merged_pickle():
    """Loads fake data from fake_dataset.pkl"""
    merged_file = "fake_dataset.pkl"

    if os.path.exists(merged_file):
        try:
            with open(merged_file, "rb") as f:
                return pickle.load(f)
        except (pickle.UnpicklingError, EOFError) as e:
            print(f"⚠️ Error loading fake_dataset.pkl: {e}")

    return {}

def load_mapping():
    """Loads existing mapping from Pickle file, ensuring proper data handling."""
    global forward_mapping, reverse_mapping
    if os.path.exists(MAPPING_FILE):
        try:
            with open(MAPPING_FILE, "rb") as f:
                data = pickle.load(f)
                forward_mapping = data.get("forward_mapping", {})
                reverse_mapping = data.get("reverse_mapping", {})
            print("📂 Existing mapping loaded.")
        except (pickle.UnpicklingError, EOFError) as e:
            print(f"⚠️ Error loading Pickle file: {e}")

def save_mapping(filename):
    """Saves the mapping to a Pickle file with filename tracking."""
    mapping_data = {
        "file_name": filename,
        "forward_mapping": forward_mapping,
        "reverse_mapping": reverse_mapping,
        "metadata": {
            "updated_at": datetime.now().isoformat(),
            "record_count": len(forward_mapping),
        },
    }
    with open(MAPPING_FILE, "wb") as f:
        pickle.dump(mapping_data, f, protocol=pickle.HIGHEST_PROTOCOL)
    print("💾 Mapping saved successfully.")

def load_and_analyze(file_path):
    """Loads the file (CSV or Excel) and returns a DataFrame."""
    if file_path.endswith(".csv"):
        return pd.read_csv(file_path, engine="python")
    elif file_path.endswith((".xls", ".xlsx")):
        return pd.read_excel(file_path)
    else:
        raise ValueError("Unsupported file format. Provide a CSV or Excel file.")

@time_it
def check_existing_mappings(input_file):
    """Checks if existing mappings match the input file and regenerates anonymized data if needed."""
    if os.path.exists(MAPPING_FILE):
        with open(MAPPING_FILE, "r") as f:
            data = json.load(f)
            stored_filename = data.get("file_name")

        if stored_filename and stored_filename == input_file:
            if not os.path.exists("anonymized.csv"):
                print("📌 Regenerating anonymized.csv from existing mappings...")
                df = load_and_analyze(input_file)
                df = apply_existing_mappings(df)
                write_file(df, "anonymized.csv")
                return True
        else:
            print("⚠️ Input file does not match stored filename. Overwriting mapping...")
    return False

def write_file(df, file_path):
    """Writes DataFrame to CSV or Excel."""
    file_ext = os.path.splitext(file_path)[-1].lower()
    if file_ext == ".csv":
        df.to_csv(file_path, index=False)
    elif file_ext in [".xls", ".xlsx"]:
        df.to_excel(file_path, index=False)
    else:
        raise ValueError("Unsupported file format. Please use CSV or Excel.")

@time_it
def analyze_column(df):
    """Uses Presidio to classify columns based on detected entity types."""
    entity_columns = {}
    for col in df.columns:
        col_lower = col.lower()
        if any(keyword in col_lower for keyword in SENSITIVE_HEADERS):
            entity_columns.setdefault("SENSITIVE", []).append(col)
            continue

        unique_values = df[col].dropna().astype(str).unique()[:10]
        entity_counts = {}

        for value in unique_values:
            if NUMERIC_PATTERN.match(value):
                continue  # Skip purely numeric values unless it's a phone/credit card

            results = analyzer.analyze(text=value, language="en")
            for result in results:
                if result.entity_type in EXCLUDED_ENTITIES:
                    continue  # Skip unwanted entities
                entity_counts[result.entity_type] = entity_counts.get(result.entity_type, 0) + 1

        if entity_counts:
            predominant_entity = max(entity_counts, key=entity_counts.get)
            if predominant_entity not in entity_columns:
                entity_columns[predominant_entity] = []
            entity_columns[predominant_entity].append(col)

    return entity_columns

def generate_fake_values(column):
    """Generates fake values while preserving uniqueness, using bothify and merged.json data."""
    col_name = column.name.lower()
    original_values = column.dropna().unique()

    match_key = None
    for key in merged_data.keys():
        if re.search(fr"\b{key}\b", col_name, re.IGNORECASE):
            match_key = key
            break

    if column.dtype.kind in 'biufc':  # Skip numeric types except IDs
        if re.search(r'\bid\b', col_name, re.IGNORECASE):
            fake_values = [f"ID-{fake.bothify(text='???##########')}" for _ in original_values]
            mapping = dict(zip(original_values, fake_values))
            forward_mapping[col_name] = mapping
            reverse_mapping[col_name] = {v: k for k, v in mapping.items()}
            return column.map(mapping).fillna(column)

        return column

    fake_values = []

    # ✅ Handle different column types
    if re.search(r'\b(phone|mobile)\b', col_name, re.IGNORECASE):
        fake_values = [fake.bothify(text='+## ##########') for _ in original_values]
    elif re.search(r'\b(card|credit)\b', col_name, re.IGNORECASE):
        fake_values = [fake.bothify(text='####-####-####-####') for _ in original_values]
    elif re.search(r'\b(domain|url|link)\b', col_name, re.IGNORECASE):
        fake_values = [
            f"www.{fake.bothify(text='?????')}/{fake.bothify(text='##??#?#?#?#?###')}/{fake.bothify(text='####')}.com"
            for _ in original_values
        ]
    elif re.search(r'\bname\b', col_name, re.IGNORECASE) and match_key:
        fake_values_list = merged_data[match_key]
        fake_values = [
            f"{name.split()[0]} {name.split()[-1]} {fake.bothify(text='????')}" if ' ' in name else f"{name} {fake.bothify(text='????')}"
            for name in fake_values_list[:len(original_values)]
        ]
    elif match_key:
        fake_values_list = merged_data[match_key]
        fake_values = [
            fake_values_list[i] if i < len(fake_values_list) else fake.bothify(text='??????????')
            for i in range(len(original_values))
        ]
    else: fake_values = [fake.bothify(text="?????##########") for _ in original_values]

    mapping = dict(zip(original_values, fake_values))
    forward_mapping[col_name] = mapping
    reverse_mapping[col_name] = {v: k for k, v in mapping.items()}

    return column.map(mapping).fillna(column)

@time_it
def mask_columnwise(df):
    """Replaces sensitive data only in columns detected as sensitive by Presidio."""
    sensitive_columns = analyze_column(df)

    for entity, cols in sensitive_columns.items():
        for column in cols:
            print(f"🔒 Masking sensitive column: {column} (Detected as {entity})")
            df[column] = generate_fake_values(df[column])

    return df

def apply_existing_mappings(df):
    """Applies stored mappings to re-anonymize a DataFrame if mappings.json exists."""
    for column in df.columns:
        col_name = column.lower()
        if col_name in forward_mapping:
            df[column] = df[column].astype(str).map(forward_mapping[col_name]).fillna(df[column])
    return df


def restore_data(df):
    """Restores original values using stored mappings."""
    for column in df.columns:
        col_name = column.lower()
        if col_name in reverse_mapping:
            df[column] = df[column].astype(str).map(reverse_mapping[col_name]).fillna(df[column])
    return df


def check_if_files_match(original_file, restored_file):
    """Checks if the original and restored files match."""
    file_ext = os.path.splitext(original_file)[-1].lower()
    original_df = pd.read_csv(original_file) if file_ext == ".csv" else pd.read_excel(original_file)
    restored_df = pd.read_csv(restored_file) if file_ext == ".csv" else pd.read_excel(restored_file)

    test = original_df.equals(restored_df)
    print(f"📊 Are files identical? {test}")
    if not test:
        print("⚠️ Files are not identical!")
    else:
        print("✅ Files match perfectly!")

if __name__ == "__main__":
    merged_data = load_merged_pickle()
    load_mapping()
    # uploaded = files.upload()
    # input_file = list(uploaded.keys())[0].strip().replace(' ', '-')
    input_file = 'smaller_100k_companies.csv'

    if not check_existing_mappings(input_file):
        df = load_and_analyze(input_file)
        df = mask_columnwise(df)
        write_file(df, "anonymized.csv")
        save_mapping(input_file)

    restored_df = restore_data(load_and_analyze("anonymized.csv"))
    write_file(restored_df, "restored.csv")
    check_if_files_match(input_file, "restored.csv")

    print("✅ Processing complete!")


⏳ Execution time check_existing_mappings: 0.000007 seconds

⏳ Execution time analyze_column: 0.789008 seconds
🔒 Masking sensitive column: id (Detected as SENSITIVE)
🔒 Masking sensitive column: name (Detected as SENSITIVE)
🔒 Masking sensitive column: domain (Detected as URL)
🔒 Masking sensitive column: linkedin url (Detected as URL)
🔒 Masking sensitive column: locality (Detected as LOCATION)
🔒 Masking sensitive column: country (Detected as LOCATION)

⏳ Execution time mask_columnwise: 15.929399 seconds
💾 Mapping saved successfully.
📊 Are files identical? True
✅ Files match perfectly!
✅ Processing complete!


In [ ]:
# Converted list of dictionary of lists into dictionary
# Converting json to pickle
import os
import json
import pickle

def convert_merged_json(input_file="merged.json", output_file="converted_merged.json"):
    """Converts a list of dictionaries into a single dictionary and saves it."""
    if not os.path.exists(input_file):
        print(f"⚠️ File {input_file} not found.")
        return

    try:
        with open(input_file, "r", encoding="utf-8") as f:
            data = json.load(f)

        if isinstance(data, list):
            merged_dict = {}
            for entry in data:
                if isinstance(entry, dict):
                    merged_dict.update(entry)

            with open(output_file, "w", encoding="utf-8") as f:
                json.dump(merged_dict, f, indent=4)

            print(f"✅ Successfully converted and saved as {output_file}")
        else:
            print("⚠️ The input JSON is already in the correct format.")

    except json.JSONDecodeError as e:
        print(f"❌ Error parsing JSON: {e}")
    except Exception as e:
        print(f"❌ Unexpected error: {e}")

def converting_json_to_pickle(input_file="fake_dataset.json", output_file="fake_dataset.pkl"):
  with open(input_file, 'r') as fp:
    data = json.load(fp)

  with open(output_file, 'wb') as fp:
    pickle.dump(data, fp)

# converting_json_to_pickle()

In [ ]:
# Testing - classify orgainization
import re
import os
import gzip
import time
import json
import spacy
import string
import random
import pandas as pd
from faker import Faker
from collections import defaultdict, deque
from presidio_analyzer import AnalyzerEngine

# Variable defined
ID = {}
fake_data={}
used_urls = set()

url_extensions = [
    ".com", ".net", ".org", ".edu", ".gov", ".co", ".us", ".uk", ".in", ".ru",
    ".jp", ".cn", ".de", ".fr", ".it", ".nl", ".es", ".br", ".au", ".ca",
    ".ch", ".se", ".no", ".za", ".mx", ".ar", ".be", ".kr", ".pl", ".tr",
    ".ua", ".ir", ".sa", ".ae", ".my", ".sg", ".hk", ".tw", ".nz", ".id",
    ".th", ".ph", ".vn", ".bd", ".lk", ".np", ".pk", ".cz", ".gr", ".hu",
    ".fi", ".dk", ".il", ".ie", ".pt", ".sk", ".si", ".ro", ".bg", ".rs",
    ".lt", ".lv", ".ee", ".hr", ".ba", ".md", ".ge", ".kz", ".by", ".tm",
    ".uz", ".af", ".qa", ".om", ".kw", ".bh", ".ye", ".jo", ".lb", ".sy",
    ".iq", ".ps", ".az", ".am", ".kg", ".mn", ".bt", ".mv", ".mm", ".kh",
    ".la", ".tl", ".sb", ".fj", ".pg", ".to", ".tv", ".ws", ".fm", ".ki"
]
with gzip.open("faker_data_v2.json.gz", "rt", encoding = 'utf-8') as f: fake_data_list = json.load(f)
# Objects
nlp = spacy.load("en_core_web_lg")
fake=Faker()
analyzer = AnalyzerEngine()

for data in fake_data_list:
    for key,value in data.items(): fake_data[key]=deque(value)
domain_pool = list(fake_data.get('url', deque()))

entity_mapping={
    'names':'PERSON',
    'emails':'EMAIL_ADDRESS',
    'phone':'PHONE_NUMBER',
    'location':'LOCATION',
    'credit':'CREDIT_CARD',
    'url':'URL',
    'country':'COUNTRY',
    'company':"ORG",
    'id':'ID',
}

mapping_file="mapping.json"
forward_mapping=defaultdict(dict)
reverse_mapping=defaultdict(dict)

if os.path.exists(mapping_file):
    with open(mapping_file, "r") as f:
        mapping_data = json.load(f)
        forward_mapping.update(mapping_data.get("forward_mapping", {}))
        reverse_mapping.update(mapping_data.get("reverse_mapping", {}))
def time_it(func):
    """Decorator to measure execution time of functions."""
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f'\n⏳ Execution time {func.__name__}: {end-start:.6f} seconds')
        return result
    return wrapper

@time_it
def analyze_column(df):
    entity_columns = {}

    # Step 1: Use Presidio to analyze all columns
    for col in df.columns:
        if 'id' in col.lower():
            entity_columns[col] = 'ID'
        elif 'country' in col.lower():
            entity_columns[col] = 'COUNTRY'
        else:
            unique_values = df[col].dropna().astype(str).unique()[:25]
            entity_counts = {}

            for value in unique_values:
                results = analyzer.analyze(text=value, language='en')
                for result in results:
                    entity_counts[result.entity_type] = entity_counts.get(result.entity_type, 0) + 1

            if entity_counts:
                predominant_entity = max(entity_counts, key=entity_counts.get)
                entity_columns[col] = predominant_entity
                if predominant_entity=="LOCATION":
                  org_count=0
                  for value in unique_values:
                    doc=nlp(value)
                    for ent in doc.ents:
                      if ent.label_=="ORG":
                        org_count+=1
                  if org_count>5:
                    predominant_entity="ORG"
                entity_columns[col]=predominant_entity

    # Step 2: Use SpaCy to analyze non-numeric and unclassified columns
    for col in df.select_dtypes(exclude=['number']).columns:
        if col not in entity_columns:
            unique_values = df[col].dropna().astype(str).unique()[:25]
            org_count = 0

            for value in unique_values:
                doc = nlp(value)
                for ent in doc.ents:
                    if ent.label_ == 'ORG':
                        org_count += 1

            # If more than half of the sample values are ORG, classify as ORG
            if org_count > 12:
                entity_columns[col] = 'ORG'

    return entity_columns


def modify_fake_value(category, base_fake_value, counter):
    if category == "names": return f"{base_fake_value} {string.ascii_uppercase[counter % 26]}."
    elif category == "emails":
        name, domain = base_fake_value.split("@")
        return f"{name}{counter}@{domain}"
    elif category in {"location", "country"}: return f"{base_fake_value}, District {counter % 100_000_000 + 1}"
    elif category == "url":
        # Check if the URL already exists, if so, append a country extension
        fake_value = base_fake_value
        while fake_value in used_urls:
            ext = random.choice(url_extensions)
            if not fake_value.endswith(ext):
                fake_value += ext
        used_urls.add(fake_value)
        return fake_value
    elif category == "phone": return f"{base_fake_value[:-2]}{counter % 100:02d}"
    elif category == "company": return f"{base_fake_value} Group {counter % 100_000_000 + 1}"
    elif category == "credit": return f"{base_fake_value[:-4]}{counter % 10000:04d}"
    else: return f"{base_fake_value}-{counter}"


def get_fake_value(category, original_value):
    global ID
    fake_value = None
    original_value = original_value.strip()
    if original_value in forward_mapping[category]: return forward_mapping[category][original_value]

    # Special case for ID
    if category == 'id':
        length = 6
        while True:
            fake_value = fake.bothify(text=f'ID-{"#"*length}')
            if fake_value not in ID:
                ID[fake_value] = True
                break
            if len(ID) >= 10 ** length: length += 1
    elif category == 'url':
        domain1, domain2 = random.sample(domain_pool, 2)
        base_fake_value = f"https://{domain1.lower()}/{domain2.lower()}.co"

        if base_fake_value not in reverse_mapping["url"]:
            fake_value = base_fake_value
        else:
            counter = len(reverse_mapping["url"])
            fake_value = modify_fake_value("url", base_fake_value, counter)

    elif fake_data.get(category):
      for _ in range(len(fake_data[category])):
          candidate = fake_data[category].popleft()
          fake_data[category].append(candidate)  # Reinsert at end (rotation)
          if candidate not in reverse_mapping[category]:
              fake_value = candidate
              break
      else:
          counter = len(reverse_mapping[category])
          base_fake_value = random.choice(list(reverse_mapping[category])) if reverse_mapping[category] else f"{category}_"
          fake_value = modify_fake_value(category, base_fake_value, counter)

    if not fake_value or fake_value in reverse_mapping[category]:
        counter = len(reverse_mapping[category])
        base = fake_value if fake_value else f"{category}_"
        fake_value = modify_fake_value(category, base, counter)

    # Store forward & reverse mappings
    forward_mapping[category][original_value] = fake_value
    reverse_mapping[category][fake_value] = original_value

    return fake_value


@time_it
def mask_dataframe(df):
    for col, entity in entity_columns.items():
        matching_keys = [key for key, value in entity_mapping.items() if value == entity]
        if matching_keys:
            df[col] = df[col].astype(str).apply(lambda x: get_fake_value(matching_keys[0], str(x)) if x else str(x))
    return df

def restore_original_value(category, fake_value):
    return reverse_mapping[category].get(fake_value, fake_value)

@time_it
def unmask_dataframe(df):
    for col, entity in entity_columns.items():
        matching_keys = [key for key, value in entity_mapping.items() if value == entity]
        if matching_keys:
            category = matching_keys[0]
            df[col] = df[col].astype(str).apply(lambda x: restore_original_value(category, str(x)) if x else str(x))

    return df
def compare_files(original_file, restored_file):
    """Check if the original and restored files are identical."""
    file_ext = os.path.splitext(original_file)[-1].lower()
    original_df = pd.read_excel(original_file) if file_ext == ".xlsx" else pd.read_csv(original_file)
    restored_df = pd.read_excel(restored_file) if file_ext == ".xlsx" else pd.read_csv(restored_file)

    is_identical = original_df.equals(restored_df)
    print(f"📊 Are files identical? {'✅ Yes' if is_identical else '❌ No'}")
    if not is_identical:
        print("⚠️ The restored file does not match the original. There may be an issue with the mapping.")

    return is_identical

def de_anonymize_paragraph(text):
  for category,mapping in reverse_mapping.items():
    for fake_value,original_value in mapping.items():
      if fake_value in text:
        text=text.replace(fake_value,original_value)
  return text

def save_mapping(filename):
    mapping_data={
        "filename":filename,
        "updated_at": time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()),
        "forward_mapping":forward_mapping,
        "reverse_mapping":reverse_mapping
    }
    with open(mapping_file, "w") as f:
        json.dump(mapping_data, f, indent=4)


if __name__=="__main__":
    input_file="smaller_100k_companies.csv"
    file_ext=os.path.splitext(input_file)[-1].lower()

    df = pd.read_excel(input_file, dtype=str) if file_ext == ".xlsx" else pd.read_csv(input_file, dtype=str, low_memory=False)
    entity_columns=analyze_column(df)
    print(entity_columns)

    anonymized_df=mask_dataframe(df)
    output_file="anonymized.xlsx" if file_ext==".xlsx" else "anonymized.csv"
    anonymized_df.to_excel(output_file,index=False) if file_ext==".xlsx" else anonymized_df.to_csv(output_file,index=False)
    print(f"✅ Anonymized data saved as {output_file}")

    save_mapping(input_file)
    restored_df=unmask_dataframe(pd.read_excel(output_file) if file_ext==".xlsx" else pd.read_csv(output_file))
    restored_file="restored.xlsx" if file_ext==".xlsx" else "restored.csv"
    restored_df.to_excel(restored_file,index=False) if file_ext==".xlsx" else restored_df.to_csv(restored_file,index=False)
    print(f"✅ Restored data saved as {restored_file}")
    compare_files(input_file, restored_file)


⏳ Execution time analyze_column: 3.669720 seconds
{'id': 'ID', 'name': 'ORG', 'domain': 'URL', 'size range': 'DATE_TIME', 'locality': 'LOCATION', 'country': 'COUNTRY', 'linkedin url': 'URL', 'current employee estimate': 'US_DRIVER_LICENSE', 'total employee estimate': 'US_DRIVER_LICENSE'}

⏳ Execution time mask_dataframe: 2.481203 seconds
✅ Anonymized data saved as anonymized.csv

⏳ Execution time unmask_dataframe: 0.367132 seconds
✅ Restored data saved as restored.csv
📊 Are files identical? ✅ Yes


In [ ]:
# Main Code - Optimizing for excel :- Main Code
import re
import os
import gzip
import time
import json
import spacy
import string
import random
import tempfile
import pandas as pd
from faker import Faker
from pathlib import Path
from collections import defaultdict, deque
from presidio_analyzer import AnalyzerEngine

# Variable defined
ID, fake_data, used_urls, entity_columns  = {}, {}, set(), {}
COMMENT_KEYWORDS = ['comments', 'description', 'remarks', 'note', 'feedback', 'observation']

url_extensions = [
    ".com", ".net", ".org", ".edu", ".gov", ".co", ".us", ".uk", ".in", ".ru",
    ".jp", ".cn", ".de", ".fr", ".it", ".nl", ".es", ".br", ".au", ".ca",
    ".ch", ".se", ".no", ".za", ".mx", ".ar", ".be", ".kr", ".pl", ".tr",
    ".ua", ".ir", ".sa", ".ae", ".my", ".sg", ".hk", ".tw", ".nz", ".id",
    ".th", ".ph", ".vn", ".bd", ".lk", ".np", ".pk", ".cz", ".gr", ".hu",
    ".fi", ".dk", ".il", ".ie", ".pt", ".sk", ".si", ".ro", ".bg", ".rs",
    ".lt", ".lv", ".ee", ".hr", ".ba", ".md", ".ge", ".kz", ".by", ".tm",
    ".uz", ".af", ".qa", ".om", ".kw", ".bh", ".ye", ".jo", ".lb", ".sy",
    ".iq", ".ps", ".az", ".am", ".kg", ".mn", ".bt", ".mv", ".mm", ".kh",
    ".la", ".tl", ".sb", ".fj", ".pg", ".to", ".tv", ".ws", ".fm", ".ki"
]
with open("test.json", "r", ) as f: fake_data_list = json.load(f)
# Objects
nlp = spacy.load("en_core_web_sm")
fake=Faker()
analyzer = AnalyzerEngine()

for data in fake_data_list:
    for key,value in data.items(): fake_data[key]=deque(value)
domain_pool = list(fake_data.get('url', deque()))

entity_mapping={
    'names':'PERSON',
    'emails':'EMAIL_ADDRESS',
    'phone':'PHONE_NUMBER',
    'location':'LOCATION',
    'credit':'CREDIT_CARD',
    'url':'URL',
    'country':'COUNTRY',
    'company':"ORG",
    'id':'ID',
}

mapping_file="mapping.json"
forward_mapping=defaultdict(dict)
reverse_mapping=defaultdict(dict)

if os.path.exists(mapping_file):
    with open(mapping_file, "r") as f:
        mapping_data = json.load(f)
        forward_mapping.update(mapping_data.get("forward_mapping", {}))
        reverse_mapping.update(mapping_data.get("reverse_mapping", {}))
# For comments logic
def get_comment_columns(df):
    return [col for col in df.columns if any(kw in col.lower() for kw in COMMENT_KEYWORDS)]

def mask_comment_columns(df, mapping):
    comment_cols = get_comment_columns(df)
    print('Inside mask comment cols:', comment_cols)
    for col in comment_cols:
        df[col] = df[col].astype(str).apply(lambda x: mask_comment_text(x, mapping))
    return df

def mask_comment_text(text, mapping):
    if not text.strip():
        return text
    doc = nlp(text)
    modified_text = text
    for token in doc:
        if token.pos_ == "PROPN" or token.ent_type_:
            mapped_value = mapping['names'].get(token.text, None)
            if mapped_value:
              modified_text = re.sub(
                rf"\b{re.escape(token.text)}(?=[\W_]|$)", mapped_value, modified_text
            )

    return modified_text

def unmask_comment_text(text, reverse_mapping):
    if not text.strip(): return text
    doc = nlp(text)
    modified_text = text
    for token in doc:
        if token.pos_ == "PROPN" or token.ent_type_:
            original_value = reverse_mapping['names'].get(token.text, None)
            if original_value:
                modified_text = re.sub(
                    rf"\b{re.escape(token.text)}(?=[\W_]|$)", original_value, modified_text
                )
    return modified_text

# To track time
def time_it(func):
    """Decorator to measure execution time of functions."""
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f'\n⏳ Execution time {func.__name__}: {end-start:.6f} seconds')
        return result
    return wrapper

@time_it
def analyze_column(df):
    # Step 1: Use Presidio to analyze all columns
    comment_cols = get_comment_columns(df)

    for col in df.columns:
        if col in comment_cols: continue

        if 'id' in col.lower():
            entity_columns[col] = 'ID'
        elif 'country' in col.lower():
            entity_columns[col] = 'COUNTRY'
        else:
            unique_values = df[col].dropna().astype(str).unique()[:25]
            entity_counts = {}

            for value in unique_values:
                results = analyzer.analyze(text=value, language='en')
                for result in results:
                    entity_counts[result.entity_type] = entity_counts.get(result.entity_type, 0) + 1

            if entity_counts:
                predominant_entity = max(entity_counts, key=entity_counts.get)
                entity_columns[col] = predominant_entity
                if predominant_entity=="LOCATION":
                  org_count=0
                  for value in unique_values:
                    doc=nlp(value)
                    for ent in doc.ents:
                      if ent.label_=="ORG":
                        org_count+=1
                  if org_count>5:
                    predominant_entity="ORG"
                entity_columns[col]=predominant_entity

    # Step 2: Use SpaCy to analyze non-numeric and unclassified columns
    for col in df.select_dtypes(exclude=['number']).columns:
        if col not in entity_columns:
            unique_values = df[col].dropna().astype(str).unique()[:25]
            org_count = 0

            for value in unique_values:
                doc = nlp(value)
                for ent in doc.ents:
                    if ent.label_ == 'ORG':
                        org_count += 1

            # If more than half of the sample values are ORG, classify as ORG
            if org_count > 12:
                entity_columns[col] = 'ORG'

    return entity_columns

def modify_fake_value(category, base_fake_value, counter):
    if category == "names": return f"{base_fake_value}{string.ascii_lowercase[counter % 26]}"
    elif category == "emails":
        name, domain = base_fake_value.split("@")
        return f"{name}{counter}@{domain}"
    elif category in {"location", "country"}: return f"{base_fake_value}, District {counter % 100_000_000 + 1}"
    elif category == "url":
        # Check if the URL already exists, if so, append a country extension
        fake_value = base_fake_value
        while fake_value in used_urls:
            ext = random.choice(url_extensions)
            if not fake_value.endswith(ext):
                fake_value += ext
        used_urls.add(fake_value)
        return fake_value
    elif category == "phone": return f"{base_fake_value[:-2]}{counter % 100:02d}"
    elif category == "company": return f"{base_fake_value} Group {counter % 100_000_000 + 1}"
    elif category == "credit": return f"{base_fake_value[:-4]}{counter % 10000:04d}"
    else: return f"{base_fake_value}-{counter}"


def get_fake_value(category, original_value):
    global ID
    fake_value = None
    if category == "names":
        original_tokens = str(original_value).split()
        fake_tokens = []
        used_fake_names = set(reverse_mapping[category])

        for token in original_tokens:
            if token in forward_mapping[category]:
                fake_token = forward_mapping[category][token]
            else:
                fake_token = None

                if fake_data.get(category):
                    for _ in range(len(fake_data[category])):
                        candidate = fake_data[category].popleft()
                        fake_data[category].append(candidate)

                        if candidate not in used_fake_names and candidate != token:
                            fake_token = candidate
                            break
                        else:
                            # Modify it if already used
                            counter = 0
                            base = candidate
                            while True:
                                modified = modify_fake_value(category, base, counter)
                                if modified not in used_fake_names and modified != token:
                                    fake_token = modified
                                    break
                                counter += 1
                            break  # once we found a modified one, skip the rest

                # Absolute fallback: no fake_data or nothing worked
                if not fake_token:
                    counter = 0
                    base_pool = list(used_fake_names)
                    random.shuffle(base_pool)

                    for base in base_pool[:10]:  # Try 10
                        fake_token = modify_fake_value(category, base, counter)
                        if fake_token not in used_fake_names and fake_token != token:
                            break
                        counter += 1
                    else:
                        # Final fallback: just keep modifying the original token
                        while True:
                            fake_token = modify_fake_value(category, token, counter)
                            if fake_token not in used_fake_names:
                                break
                            counter += 1

                # Save mappings
                forward_mapping[category][token] = fake_token
                reverse_mapping[category][fake_token] = token
                used_fake_names.add(fake_token)

            fake_tokens.append(fake_token)

        return " ".join(fake_tokens)



    if original_value in forward_mapping[category]: return forward_mapping[category][original_value]
    # Special case for ID
    if category == 'id':
        length = 6
        while True:
            fake_value = fake.bothify(text=f'ID-{"#"*length}')
            if fake_value not in ID:
                ID[fake_value] = True
                break
            if len(ID) >= 10 ** length: length += 1
    elif category == 'url':
        domain1, domain2 = random.sample(domain_pool, 2)
        base_fake_value = f"https://{domain1.lower()}/{domain2.lower()}.co"

        if base_fake_value not in reverse_mapping["url"]:
            fake_value = base_fake_value
        else:
            counter = len(reverse_mapping["url"])
            fake_value = modify_fake_value("url", base_fake_value, counter)

    elif fake_data.get(category):
      length = len(fake_data[category])
      for _ in range(length):
            candidate = fake_data[category].popleft()
            fake_data[category].append(candidate)  # Reinsert at end (rotation)
            if candidate not in reverse_mapping[category]:
                fake_value = candidate
                break
      else:
          counter = len(reverse_mapping[category])
          base_fake_value = (
              random.choice(list(reverse_mapping[category]))
              if reverse_mapping[category]
              else f"{category}_"
          )
          fake_value = modify_fake_value(category, base_fake_value, counter)

    if not fake_value or fake_value in reverse_mapping[category]:
        counter = len(reverse_mapping[category])
        base = fake_value if fake_value else f"{category}_"
        fake_value = modify_fake_value(category, base, counter)

    # Store forward & reverse mappings
    forward_mapping[category][original_value] = fake_value
    reverse_mapping[category][fake_value] = original_value

    return fake_value


@time_it
def mask_dataframe(df):
    for col, entity in entity_columns.items():
        matching_keys = [
            key for key, value in entity_mapping.items() if value == entity
        ]
        if matching_keys:
            category = matching_keys[0]
            df[col] = df[col].astype(str).apply(
                lambda x: get_fake_value(category, x) if x else x
            )
    return df


def restore_original_value(category, fake_value):
    return reverse_mapping[category].get(fake_value, fake_value)

@time_it
def unmask_dataframe(df):
    for col, entity in entity_columns.items():
        matching_keys = [key for key, value in entity_mapping.items() if value == entity]
        if matching_keys:
            category = matching_keys[0]

            if category == "names":
                def reverse_name(val):
                    tokens = str(val).split()
                    original_tokens = [reverse_mapping[category].get(tok, tok) for tok in tokens]
                    return " ".join(original_tokens)
                df[col] = df[col].astype(str).apply(lambda x: reverse_name(x) if pd.notna(x) else x)

            else:
                df[col] = df[col].astype(str).apply(
                    lambda x: restore_original_value(category, str(x)) if pd.notna(str(x)) else x
                )
    comment_cols = get_comment_columns(df)
    for col in comment_cols:
        df[col] = df[col].astype(str).apply(lambda x: unmask_comment_text(x, reverse_mapping))
    return df

@time_it
def compare_files(original_csv, restored_csv):
    """Compare two CSV files and return if they are identical."""
    original_df = pd.read_csv(original_csv, dtype=str).sort_index(axis=1).reset_index(drop=True)
    restored_df = pd.read_csv(restored_csv, dtype=str).sort_index(axis=1).reset_index(drop=True)

    is_identical = original_df.equals(restored_df)
    print(f"\n📊 Are files identical? {'✅ Yes' if is_identical else '❌ No'}")

    if not is_identical:
        print("⚠️ The restored file does not match the original. Investigate the mapping or masking logic.")

    return is_identical

def de_anonymize_paragraph(text):
  for category,mapping in reverse_mapping.items():
    for fake_value,original_value in mapping.items():
      if fake_value in text:
        text=text.replace(fake_value,original_value)
  return text

def save_mapping(filename):
    mapping_data={
        "filename":filename,
        "updated_at": time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()),
        "forward_mapping":forward_mapping,
        "reverse_mapping":reverse_mapping
    }
    with open(mapping_file, "a") as f:
        json.dump(mapping_data, f, indent=4)

def process_file(input_file, file_ext):
    input_base = os.path.splitext(os.path.basename(input_file))[0]
    output_dir = os.path.join(".", input_base)

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    if file_ext == ".xlsx":
        xl = pd.read_excel(input_file, sheet_name=None, dtype=str, engine='openpyxl')

        for sheet_name, df in xl.items():
            print(f"\n🔍 Processing sheet: {sheet_name}")

            original_csv = os.path.join(output_dir, f"{sheet_name}_original.csv")
            masked_csv = os.path.join(output_dir, f"{sheet_name}_masked.csv")
            restored_csv = os.path.join(output_dir, f"{sheet_name}_restored.csv")

            df.to_csv(original_csv, index=False)

            entity_columns = analyze_column(df)
            print(f"Detected entities: {entity_columns}")

            masked_df = mask_dataframe(df.copy())
            # masked_df = mask_comment_columns(masked_df, forward_mapping)
            restored_df = unmask_dataframe(masked_df.copy())

            masked_df.to_csv(masked_csv, index=False)
            restored_df.to_csv(restored_csv, index=False)

            compare_files(original_csv, restored_csv)

            save_mapping(f'{input_file}-->{sheet_name}')
            entity_columns.clear()


    elif file_ext == ".csv":
        df = pd.read_csv(input_file, dtype=str, low_memory=False)
        print(f"\n🔍 Processing CSV: {input_file}")

        original_csv = os.path.join(output_dir, "original.csv")
        masked_csv = os.path.join(output_dir, "anonymized.csv")
        restored_csv = os.path.join(output_dir, "restored.csv")

        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        df.to_csv(original_csv, index=False)

        entity_columns = analyze_column(df)
        print(f"Detected entities: {entity_columns}")

        masked_df = mask_dataframe(df.copy())
        # masked_df = mask_comment_columns(masked_df, forward_mapping)
        restored_df = unmask_dataframe(masked_df.copy())

        masked_df.to_csv(masked_csv, index=False)
        restored_df.to_csv(restored_csv, index=False)

        compare_files(original_csv, restored_csv)

        save_mapping(input_file)
    else:
        raise ValueError("Unsupported file type. Use .csv or .xlsx only.")


if __name__ == "__main__":
    input_file = "for_colab_test.csv"
    file_ext = os.path.splitext(input_file)[-1].lower()
    process_file(input_file, file_ext)


🔍 Processing CSV: for_colab_test.csv

⏳ Execution time analyze_column: 2.157893 seconds
Detected entities: {'names': 'PERSON', 'emails': 'URL', 'phone': 'PHONE_NUMBER', 'credit': 'US_BANK_NUMBER', 'url': 'URL', 'location': 'LOCATION', 'company': 'ORG'}

⏳ Execution time mask_dataframe: 0.005189 seconds

⏳ Execution time unmask_dataframe: 0.004125 seconds

📊 Are files identical? ❌ No
⚠️ The restored file does not match the original. Investigate the mapping or masking logic.

⏳ Execution time compare_files: 0.007497 seconds


In [ ]:
# Main Code - Testing comments :- Main Code
import re
import os
import gzip
import time
import json
import spacy
import string
import random
import tempfile
import pandas as pd
from faker import Faker
from pathlib import Path
from collections import defaultdict, deque
from presidio_analyzer import AnalyzerEngine

# Variable defined
ID, fake_data, used_urls, entity_columns  = {}, {}, set(), {}
COMMENT_KEYWORDS = ['comments', 'description', 'remarks', 'note', 'feedback', 'observation']

url_extensions = [
    ".com", ".net", ".org", ".edu", ".gov", ".co", ".us", ".uk", ".in", ".ru",
    ".jp", ".cn", ".de", ".fr", ".it", ".nl", ".es", ".br", ".au", ".ca",
    ".ch", ".se", ".no", ".za", ".mx", ".ar", ".be", ".kr", ".pl", ".tr",
    ".ua", ".ir", ".sa", ".ae", ".my", ".sg", ".hk", ".tw", ".nz", ".id",
    ".th", ".ph", ".vn", ".bd", ".lk", ".np", ".pk", ".cz", ".gr", ".hu",
    ".fi", ".dk", ".il", ".ie", ".pt", ".sk", ".si", ".ro", ".bg", ".rs",
    ".lt", ".lv", ".ee", ".hr", ".ba", ".md", ".ge", ".kz", ".by", ".tm",
    ".uz", ".af", ".qa", ".om", ".kw", ".bh", ".ye", ".jo", ".lb", ".sy",
    ".iq", ".ps", ".az", ".am", ".kg", ".mn", ".bt", ".mv", ".mm", ".kh",
    ".la", ".tl", ".sb", ".fj", ".pg", ".to", ".tv", ".ws", ".fm", ".ki"
]
with open("test.json", "r", ) as f: fake_data_list = json.load(f)
# Objects
nlp = spacy.load("en_core_web_sm")
fake=Faker()
analyzer = AnalyzerEngine()

for data in fake_data_list:
    for key,value in data.items(): fake_data[key]=deque(value)
domain_pool = list(fake_data.get('url', deque()))

entity_mapping={
    'names':'PERSON',
    'emails':'EMAIL_ADDRESS',
    'phone':'PHONE_NUMBER',
    'location':'LOCATION',
    'credit':'CREDIT_CARD',
    'url':'URL',
    'country':'COUNTRY',
    'company':"ORG",
    'id':'ID',
}

mapping_file="mapping.json"
forward_mapping=defaultdict(dict)
reverse_mapping=defaultdict(dict)

if os.path.exists(mapping_file):
    with open(mapping_file, "r") as f:
        mapping_data = json.load(f)
        forward_mapping.update(mapping_data.get("forward_mapping", {}))
        reverse_mapping.update(mapping_data.get("reverse_mapping", {}))

# To track time
def time_it(func):
    """Decorator to measure execution time of functions."""
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f'\n⏳ Execution time {func.__name__}: {end-start:.6f} seconds')
        return result
    return wrapper

# For comments logic
def get_comment_columns(df):
    return [col for col in df.columns if any(kw in col.lower() for kw in COMMENT_KEYWORDS)]

@time_it
def mask_comment_columns(df, mapping):
    comment_cols = get_comment_columns(df)
    print('Inside mask comment cols:', comment_cols)
    for col in comment_cols:
        df[col] = df[col].astype(str).apply(lambda x: mask_comment_text(x, mapping))
    return df

def mask_comment_text(text, mapping):
    if not text.strip():
        return text
    doc = nlp(text)
    modified_text = text
    for token in doc:
        if token.pos_ == "PROPN" or token.ent_type_:
            mapped_value = mapping['names'].get(token.text, token.text)
            if mapped_value:
              modified_text = re.sub(
                rf"\b{re.escape(token.text)}(?=[\W_]|$)", mapped_value, modified_text
            )

    return modified_text

def unmask_comment_text(text, reverse_mapping):
    if not text.strip():
        return text

    doc = nlp(text)
    replacements = {}  # Collect replacements first

    for token in doc:
        if token.pos_ == "PROPN" or token.ent_type_:
            original_value = reverse_mapping['names'].get(token.text, token.text)
            if original_value:
                replacements[token.text] = original_value

    for fake_token, real_token in replacements.items():
        text = re.sub(
            rf"\b{re.escape(fake_token)}(?=[\W_]|$)", real_token, text
        )

    return text

@time_it
def analyze_column(df):
    # Step 1: Use Presidio to analyze all columns
    comment_cols = get_comment_columns(df)

    for col in df.columns:
        if col in comment_cols: continue

        if 'id' in col.lower():
            entity_columns[col] = 'ID'
        elif 'country' in col.lower():
            entity_columns[col] = 'COUNTRY'
        else:
            unique_values = df[col].dropna().astype(str).unique()[:25]
            entity_counts = {}

            for value in unique_values:
                results = analyzer.analyze(text=value, language='en')
                for result in results:
                    entity_counts[result.entity_type] = entity_counts.get(result.entity_type, 0) + 1

            if entity_counts:
                predominant_entity = max(entity_counts, key=entity_counts.get)
                entity_columns[col] = predominant_entity
                if predominant_entity=="LOCATION":
                  org_count=0
                  for value in unique_values:
                    doc=nlp(value)
                    for ent in doc.ents:
                      if ent.label_=="ORG":
                        org_count+=1
                  if org_count>5:
                    predominant_entity="ORG"
                entity_columns[col]=predominant_entity

    # Step 2: Use SpaCy to analyze non-numeric and unclassified columns
    for col in df.select_dtypes(exclude=['number']).columns:
        if col not in entity_columns:
            unique_values = df[col].dropna().astype(str).unique()[:25]
            org_count = 0

            for value in unique_values:
                doc = nlp(value)
                for ent in doc.ents:
                    if ent.label_ == 'ORG':
                        org_count += 1

            # If more than half of the sample values are ORG, classify as ORG
            if org_count > 12:
                entity_columns[col] = 'ORG'

    return entity_columns

def modify_fake_value(category, base_fake_value, counter):
    if category == "names": return f"{base_fake_value}{string.ascii_lowercase[counter % 26]}"
    elif category == "emails":
        name, domain = base_fake_value.split("@")
        return f"{name}{counter}@{domain}"
    elif category in {"location", "country"}: return f"{base_fake_value}, District {counter % 100_000_000 + 1}"
    elif category == "url":
        # Check if the URL already exists, if so, append a country extension
        fake_value = base_fake_value
        while fake_value in used_urls:
            ext = random.choice(url_extensions)
            if not fake_value.endswith(ext):
                fake_value += ext
        used_urls.add(fake_value)
        return fake_value
    elif category == "phone": return f"{base_fake_value[:-2]}{counter % 100:02d}"
    elif category == "company": return f"{base_fake_value} Group {counter % 100_000_000 + 1}"
    elif category == "credit": return f"{base_fake_value[:-4]}{counter % 10000:04d}"
    else: return f"{base_fake_value}-{counter}"


def get_fake_value(category, original_value):
    global ID
    fake_value = None
    if category == "names":
        original_tokens = str(original_value).split()
        fake_tokens = []
        used_fake_names = set(reverse_mapping[category])

        for token in original_tokens:
            if token in forward_mapping[category]:
                fake_token = forward_mapping[category][token]
            else:
                fake_token = None

                if fake_data.get(category):
                    for _ in range(len(fake_data[category])):
                        candidate = fake_data[category].popleft()
                        fake_data[category].append(candidate)

                        if candidate not in used_fake_names and candidate != token:
                            fake_token = candidate
                            break
                        else:
                            # Modify it if already used
                            counter = 0
                            base = candidate
                            while True:
                                modified = modify_fake_value(category, base, counter)
                                if modified not in used_fake_names and modified != token:
                                    fake_token = modified
                                    break
                                counter += 1
                            break  # once we found a modified one, skip the rest

                # Absolute fallback: no fake_data or nothing worked
                if not fake_token:
                    counter = 0
                    base_pool = list(used_fake_names)
                    random.shuffle(base_pool)

                    for base in base_pool[:10]:  # Try 10
                        fake_token = modify_fake_value(category, base, counter)
                        if fake_token not in used_fake_names and fake_token != token:
                            break
                        counter += 1
                    else:
                        # Final fallback: just keep modifying the original token
                        while True:
                            fake_token = modify_fake_value(category, token, counter)
                            if fake_token not in used_fake_names:
                                break
                            counter += 1

                # Save mappings
                forward_mapping[category][token] = fake_token
                reverse_mapping[category][fake_token] = token
                used_fake_names.add(fake_token)

            fake_tokens.append(fake_token)

        return " ".join(fake_tokens)



    if original_value in forward_mapping[category]: return forward_mapping[category][original_value]
    # Special case for ID
    if category == 'id':
        length = 6
        while True:
            fake_value = fake.bothify(text=f'ID-{"#"*length}')
            if fake_value not in ID:
                ID[fake_value] = True
                break
            if len(ID) >= 10 ** length: length += 1
    elif category == 'url':
        domain1, domain2 = random.sample(domain_pool, 2)
        base_fake_value = f"https://{domain1.lower()}/{domain2.lower()}.co"

        if base_fake_value not in reverse_mapping["url"]:
            fake_value = base_fake_value
        else:
            counter = len(reverse_mapping["url"])
            fake_value = modify_fake_value("url", base_fake_value, counter)

    elif fake_data.get(category):
      length = len(fake_data[category])
      for _ in range(length):
            candidate = fake_data[category].popleft()
            fake_data[category].append(candidate)  # Reinsert at end (rotation)
            if candidate not in reverse_mapping[category]:
                fake_value = candidate
                break
      else:
          counter = len(reverse_mapping[category])
          base_fake_value = (
              random.choice(list(reverse_mapping[category]))
              if reverse_mapping[category]
              else f"{category}_"
          )
          fake_value = modify_fake_value(category, base_fake_value, counter)

    if not fake_value or fake_value in reverse_mapping[category]:
        counter = len(reverse_mapping[category])
        base = fake_value if fake_value else f"{category}_"
        fake_value = modify_fake_value(category, base, counter)

    # Store forward & reverse mappings
    forward_mapping[category][original_value] = fake_value
    reverse_mapping[category][fake_value] = original_value

    return fake_value


@time_it
def mask_dataframe(df):
    for col, entity in entity_columns.items():
        matching_keys = [
            key for key, value in entity_mapping.items() if value == entity
        ]
        if matching_keys:
            category = matching_keys[0]
            df[col] = df[col].astype(str).apply(
                lambda x: get_fake_value(category, x) if x else x
            )
    return df


def restore_original_value(category, fake_value):
    return reverse_mapping[category].get(fake_value, fake_value)

@time_it
def unmask_dataframe(df):
    for col, entity in entity_columns.items():
        matching_keys = [key for key, value in entity_mapping.items() if value == entity]
        if matching_keys:
            category = matching_keys[0]

            if category == "names":
                def reverse_name(val):
                    tokens = str(val).split()
                    original_tokens = [reverse_mapping[category].get(tok, tok) for tok in tokens]
                    return " ".join(original_tokens)
                df[col] = df[col].astype(str).apply(lambda x: reverse_name(x) if pd.notna(x) else x)

            else:
                df[col] = df[col].astype(str).apply(
                    lambda x: restore_original_value(category, str(x)) if pd.notna(str(x)) else x
                )
    comment_cols = get_comment_columns(df)
    for col in comment_cols:
        df[col] = df[col].astype(str).apply(lambda x: unmask_comment_text(x, reverse_mapping))
    return df

@time_it
def compare_files(original_csv, restored_csv):
    """Compare two CSV files and return if they are identical."""
    original_df = pd.read_csv(original_csv, dtype=str).sort_index(axis=1).reset_index(drop=True)
    restored_df = pd.read_csv(restored_csv, dtype=str).sort_index(axis=1).reset_index(drop=True)

    is_identical = original_df.equals(restored_df)
    print(f"\n📊 Are files identical? {'✅ Yes' if is_identical else '❌ No'}")

    if not is_identical:
        print("⚠️ The restored file does not match the original. Investigate the mapping or masking logic.")

    return is_identical

def de_anonymize_paragraph(text):
  for category,mapping in reverse_mapping.items():
    for fake_value,original_value in mapping.items():
      if fake_value in text:
        text=text.replace(fake_value,original_value)
  return text

def save_mapping(filename):
    mapping_data={
        "filename":filename,
        "updated_at": time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()),
        "forward_mapping":forward_mapping,
        "reverse_mapping":reverse_mapping
    }
    with open(mapping_file, "a") as f:
        json.dump(mapping_data, f, indent=4)

def process_file(input_file, file_ext):
    input_base = os.path.splitext(os.path.basename(input_file))[0]
    output_dir = os.path.join(".", input_base)

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    if file_ext == ".xlsx":
        xl = pd.read_excel(input_file, sheet_name=None, dtype=str, engine='openpyxl')

        for sheet_name, df in xl.items():
            print(f"\n🔍 Processing sheet: {sheet_name}")

            original_csv = os.path.join(output_dir, f"{sheet_name}_original.csv")
            masked_csv = os.path.join(output_dir, f"{sheet_name}_masked.csv")
            restored_csv = os.path.join(output_dir, f"{sheet_name}_restored.csv")

            df.to_csv(original_csv, index=False)

            entity_columns = analyze_column(df)
            print(f"Detected entities: {entity_columns}")

            masked_df = mask_dataframe(df.copy())
            masked_df = mask_comment_columns(masked_df, forward_mapping)
            restored_df = unmask_dataframe(masked_df.copy())

            masked_df.to_csv(masked_csv, index=False)
            restored_df.to_csv(restored_csv, index=False)

            compare_files(original_csv, restored_csv)

            save_mapping(f'{input_file}-->{sheet_name}')
            entity_columns.clear()


    elif file_ext == ".csv":
        df = pd.read_csv(input_file, dtype=str, low_memory=False)
        print(f"\n🔍 Processing CSV: {input_file}")

        original_csv = os.path.join(output_dir, "original.csv")
        masked_csv = os.path.join(output_dir, "anonymized.csv")
        restored_csv = os.path.join(output_dir, "restored.csv")

        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        df.to_csv(original_csv, index=False)

        entity_columns = analyze_column(df)
        print(f"Detected entities: {entity_columns}")

        masked_df = mask_dataframe(df.copy())
        masked_df = mask_comment_columns(masked_df, forward_mapping)
        restored_df = unmask_dataframe(masked_df.copy())

        masked_df.to_csv(masked_csv, index=False)
        restored_df.to_csv(restored_csv, index=False)

        compare_files(original_csv, restored_csv)

        save_mapping(input_file)
    else:
        raise ValueError("Unsupported file type. Use .csv or .xlsx only.")


if __name__ == "__main__":
    input_file = "for_colab_test.csv"
    file_ext = os.path.splitext(input_file)[-1].lower()
    process_file(input_file, file_ext)


🔍 Processing CSV: for_colab_test.csv

⏳ Execution time analyze_column: 1.643175 seconds
Detected entities: {'names': 'PERSON', 'emails': 'URL', 'phone': 'PHONE_NUMBER', 'credit': 'US_BANK_NUMBER', 'url': 'URL', 'location': 'LOCATION', 'company': 'ORG'}

⏳ Execution time mask_dataframe: 0.004188 seconds
Inside mask comment cols: ['comments']

⏳ Execution time mask_comment_columns: 0.480729 seconds

⏳ Execution time unmask_dataframe: 0.442396 seconds

📊 Are files identical? ❌ No
⚠️ The restored file does not match the original. Investigate the mapping or masking logic.

⏳ Execution time compare_files: 0.005113 seconds


In [ ]:
# Testing different mapping technique
# Testing - classify orgainization
# Problematic -> can't use to demask paragraph
import re
import os
import gzip
import time
import json
import spacy
import string
import random
import pandas as pd
from faker import Faker
from collections import defaultdict, deque
from presidio_analyzer import AnalyzerEngine


ID = {}
fake_data={}
nlp = spacy.load("en_core_web_lg")
fake=Faker()
analyzer = AnalyzerEngine()

with gzip.open("faker_data.json.gz", "rt", encoding='utf-8') as f:
    fake_data_list = json.load(f)
for data in fake_data_list:
    for key,value in data.items():
        fake_data[key]=deque(value)

entity_mapping={
    'names':'PERSON',
    'emails':'EMAIL_ADDRESS',
    'phone':'PHONE_NUMBER',
    'location':'LOCATION',
    'credit':'CREDIT_CARD',
    'url':'URL',
    'country':'COUNTRY',
    'company':"ORG",
    'id':'ID',
}

mapping_file="mapping.json"
forward_mapping=defaultdict(dict)
reverse_mapping=defaultdict(dict)

if os.path.exists(mapping_file):
    with open(mapping_file, "r") as f:
        mapping_data = json.load(f)
        forward_mapping.update(mapping_data.get("forward_mapping", {}))
        reverse_mapping.update(mapping_data.get("reverse_mapping", {}))
def time_it(func):
    """Decorator to measure execution time of functions."""
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f'\n⏳ Execution time {func.__name__}: {end-start:.6f} seconds')
        return result
    return wrapper

@time_it
def analyze_column(df):
    entity_columns = {}

    # Step 1: Use Presidio to analyze all columns
    for col in df.columns:
        if 'id' in col.lower():
            entity_columns[col] = 'ID'
        elif 'country' in col.lower():
            entity_columns[col] = 'COUNTRY'
        else:
            unique_values = df[col].dropna().astype(str).unique()[:25]
            entity_counts = {}

            for value in unique_values:
                results = analyzer.analyze(text=value, language='en')
                for result in results:
                    entity_counts[result.entity_type] = entity_counts.get(result.entity_type, 0) + 1

            if entity_counts:
                predominant_entity = max(entity_counts, key=entity_counts.get)
                entity_columns[col] = predominant_entity
                if predominant_entity=="LOCATION":
                  org_count=0
                  for value in unique_values:
                    doc=nlp(value)
                    for ent in doc.ents:
                      if ent.label_=="ORG":
                        org_count+=1
                  if org_count>5:
                    predominant_entity="ORG"
                entity_columns[col]=predominant_entity

    # Step 2: Use SpaCy to analyze non-numeric and unclassified columns
    for col in df.select_dtypes(exclude=['number']).columns:
        if col not in entity_columns:
            unique_values = df[col].dropna().astype(str).unique()[:25]
            org_count = 0

            for value in unique_values:
                doc = nlp(value)
                for ent in doc.ents:
                    if ent.label_ == 'ORG':
                        org_count += 1

            # If more than half of the sample values are ORG, classify as ORG
            if org_count > 12:
                entity_columns[col] = 'ORG'

    return entity_columns


def modify_fake_value(category, base_fake_value, counter):
    if category == "names": return f"{base_fake_value} {string.ascii_uppercase[counter % 26]}."
    elif category == "emails":
        name, domain = base_fake_value.split("@")
        return f"{name}{counter}@{domain}"
    elif category == "location": return f"{base_fake_value}, District {counter % 100_000_000 + 1}"
    elif category == "url": return base_fake_value.replace("://", f"://sub{counter}.", 1)
    elif category == "phone": return f"{base_fake_value[:-2]}{counter % 100:02d}"
    elif category == "company": return f"{base_fake_value} Group {counter % 50}"
    elif category == "credit": return f"{base_fake_value[:-4]}{counter % 10000:04d}"
    else: return f"{base_fake_value}-{counter}"


def get_fake_value(category, original_value, colname):
    global ID
    # Added
    if colname not in forward_mapping[category]: forward_mapping[category][colname] = {}
    if colname not in reverse_mapping[category]: reverse_mapping[category][colname] = {}
    if colname in forward_mapping[category] and original_value in forward_mapping[category][colname]:
      return forward_mapping[category][colname][original_value]

    if category == 'id':
      length = 6
      while True:
          fake_value = fake.bothify(text=f'ID-{"#"*length}')
          if fake_value not in ID:
              ID[fake_value] = True
              break
          # If exhausted all possibilities for the current length, increase the length
          if len(ID) >= 10 ** length:
              length += 1
    elif fake_data.get(category):
        fake_value = fake_data[category].pop() if fake_data[category] else None
    else:
        base_fake=random.choice(list(forward_mapping[category][colname].values()))
        if base_fake:
          counter=len(forward_mapping[category][colname])
          fake_value=modify_fake_value(category,base_fake,counter)

    forward_mapping[category][colname][original_value] = fake_value
    reverse_mapping[category][colname][fake_value] = original_value


    return fake_value

@time_it
def mask_dataframe(df):
    for col, entity in entity_columns.items():
        matching_keys = [key for key, value in entity_mapping.items() if value == entity]
        if matching_keys:
            df[col] = df[col].astype(str).apply(lambda x: get_fake_value(matching_keys[0], str(x), col) if x else str(x))
    return df

def restore_original_value(category, fake_value, colname):
    if colname in reverse_mapping[category]:
      return reverse_mapping[category][colname].get(fake_value, fake_value)
    return fake_value

@time_it
def unmask_dataframe(df):
    for col, entity in entity_columns.items():
        matching_keys = [key for key, value in entity_mapping.items() if value == entity]

        if matching_keys:
            category = matching_keys[0]
            df[col] = df[col].astype(str).apply(lambda x: restore_original_value(category, str(x), col) if x else str(x))

    return df
def compare_files(original_file, restored_file):
    """Check if the original and restored files are identical."""
    file_ext = os.path.splitext(original_file)[-1].lower()
    original_df = pd.read_excel(original_file) if file_ext == ".xlsx" else pd.read_csv(original_file)
    restored_df = pd.read_excel(restored_file) if file_ext == ".xlsx" else pd.read_csv(restored_file)

    is_identical = original_df.equals(restored_df)
    print(f"📊 Are files identical? {'✅ Yes' if is_identical else '❌ No'}")
    if not is_identical:
        print("⚠️ The restored file does not match the original. There may be an issue with the mapping.")

    return is_identical

def de_anonymize_paragraph(text):
  for category, col_map in reverse_mapping.items():
    for colname, mapping in col_map.items():
        for fake_value, original_value in mapping.items():
            if fake_value in text:
                text = text.replace(fake_value, original_value)
  return text

def save_mapping(filename):
    mapping_data={
        "filename":filename,
        "updated_at": time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()),
        "forward_mapping":dict(forward_mapping),
        "reverse_mapping":dict(reverse_mapping)
    }
    with open(mapping_file, "w") as f:
        json.dump(mapping_data, f, indent=4)

if __name__=="__main__":
    input_file="smaller_100k_companies.csv"
    file_ext=os.path.splitext(input_file)[-1].lower()

    df = pd.read_excel(input_file, dtype=str) if file_ext == ".xlsx" else pd.read_csv(input_file, dtype=str, low_memory=False)
    entity_columns=analyze_column(df)
    print(entity_columns)

    anonymized_df=mask_dataframe(df)
    output_file="anonymized.xlsx" if file_ext==".xlsx" else "anonymized.csv"
    anonymized_df.to_excel(output_file,index=False) if file_ext==".xlsx" else anonymized_df.to_csv(output_file,index=False)
    print(f"✅ Anonymized data saved as {output_file}")

    save_mapping(input_file)
    restored_df=unmask_dataframe(pd.read_excel(output_file) if file_ext==".xlsx" else pd.read_csv(output_file))
    restored_file="restored.xlsx" if file_ext==".xlsx" else "restored.csv"
    restored_df.to_excel(restored_file,index=False) if file_ext==".xlsx" else restored_df.to_csv(restored_file,index=False)
    print(f"✅ Restored data saved as {restored_file}")
    compare_files(input_file, restored_file)


⏳ Execution time analyze_column: 3.097933 seconds
{'id': 'ID', 'name': 'ORG', 'domain': 'URL', 'size range': 'DATE_TIME', 'locality': 'LOCATION', 'country': 'COUNTRY', 'linkedin url': 'URL', 'current employee estimate': 'US_DRIVER_LICENSE', 'total employee estimate': 'US_DRIVER_LICENSE'}

⏳ Execution time mask_dataframe: 2.705842 seconds
✅ Anonymized data saved as anonymized.csv

⏳ Execution time unmask_dataframe: 0.398932 seconds
✅ Restored data saved as restored.csv
📊 Are files identical? ✅ Yes


In [ ]:
"""Generating fake names and emails and comparing them"""
from itertools import count
import json
import faker
import random
from collections import Counter

fake = faker.Faker()

def generate_fake_names():
    unique_names = set()

    for _ in range(500_000):
        fake_name = fake.name()
        if fake_name in unique_names: fake_name += f' {fake.name()}'
        unique_names.add(fake_name)

    with open('fake_names.json', 'w') as fp:
        json.dump(
            {
                'names': list(unique_names)
            },
            fp,
            indent=4
        )
    print('📈Done generating fake names!')

def generate_fake_email():
    unique_emails = set()
    mail_endings = [
                    'gmail.com', 'hotmail.com',
                    'yahoo.com', 'outlook.com',
                    'live.com', 'yandex.com',
                    'mail.com', 'aol.com', 'edu.com'
                ]
    for _ in range(500_000):
        fake_email = fake.email()
        while fake_email in unique_emails:
            body, domain = fake_email.split('@')
            fake_email = f'{body}.{fake.name().replace(" ", ".")}.{fake.bothify(text="#####")}@{random.choice(mail_endings)}'
        unique_emails.add(fake_email)

    with open('fake_emails.json', 'w') as fp:
        json.dump(
            {
                'emails': list(unique_emails),
            },
            fp,
            indent=4
        )

def find_repeating_values():
    file_name = 'fake_emails.json'
    with open(file_name, 'r') as fp:
        data = json.load(fp)

    values = data.get('emails', []) or data.get('names', [])

    counts = Counter(values)
    repeating_values = {item:count for item, count in counts.items() if count > 1}

    print(f'Repeating values:{len(repeating_values)}')

# find_repeating_values()
# generate_fake_email()
# generate_fake_names()

In [ ]:
# Dummy
json_data = {
    "name": "James Bat Bond",
    "email":[ "jamesbatbond@ust.com", 'kirant700@gmail.com', '12312@ust.com'],
    "backup_email": "b20cs110@mace.ac.in",
    "phone": "+973 30982167",
    "location": "Wakanda, Africa",
    "description": "Abram is a software engineer from Wakanda.",
    "metadata": {
        "emergency_contact": "+91 9678785654",
        "company": "UST Global",
        "address": {
            "city": "New York",
            "country": "USA"
        }
    }
}

In [ ]:
# Uploading json file and doing NER on attributes
import json
import time
from google.colab import files
from presidio_analyzer import AnalyzerEngine
from presidio_anonymizer import AnonymizerEngine

analyzer = AnalyzerEngine()
anonymizer = AnonymizerEngine()

def time_it(func):
  def wrapper(*args, **kwargs):
    start = time.time()
    result = func(*args, **kwargs)
    end = time.time()
    print(f'\n⏳ Execution time {func.__name__}: {end-start:.6f} seconds')
    return result
  return wrapper

# Processing JSON Spacey
def processing_json(data):
  if isinstance(data, dict):
    return {key: processing_json(value) for key, value in data.items()}
  elif isinstance(data, list):
    return [processing_json(item) for item in data]
  elif isinstance(data, str):
    result = analyzer.analyze(
        text = data,
        entities = ['PHONE_NUMBER', 'ADDRESS', 'LOCATION', 'PERSON', 'EMAIL_ADDRESS', 'STREET_ADDRESS', 'CREDIT_CARD'],
        language = 'en'
    )
    return anonymizer.anonymize(text=data, analyzer_results=result).text if result else data
  else: return data

@time_it
def json_format_anonymized(json_data):
  processed_data = processing_json(json_data)
  print(json.dumps(processed_data, indent = 4))

def json_file_anonymized():
  file_upload = files.upload()
  for filename in file_upload.keys():
    if filename.endswith('.json'):
      with open(filename, 'r') as fp:
        json_data = json.load(fp)
        processing_json(json_data)

json_format_anonymized(json_data)

{
    "name": "<PERSON>",
    "email": [
        "<EMAIL_ADDRESS>",
        "<EMAIL_ADDRESS>",
        "<EMAIL_ADDRESS>"
    ],
    "backup_email": "<EMAIL_ADDRESS>",
    "phone": "<PHONE_NUMBER>",
    "location": "<LOCATION>, <LOCATION>",
    "description": "<PERSON> is a software engineer from <LOCATION>.",
    "metadata": {
        "emergency_contact": "<PHONE_NUMBER>",
        "company": "UST Global",
        "address": {
            "city": "<LOCATION>",
            "country": "<LOCATION>"
        }
    }
}

⏳ Execution time json_format_anonymized: 0.152774 seconds


In [1]:
!pip install spacy
!pip install faker
!pip install presidio_analyzer
!python -m spacy download en_core_web_sm
!pip install --upgrade --force-reinstall "Cython" "spacy"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 102.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
# Main Code - Fixing demasking of comments :- Main Code
import re
import os
import gzip
import time
import json
import spacy
import string
import random
import tempfile
import pandas as pd
from faker import Faker
from pathlib import Path
from collections import defaultdict, deque
from presidio_analyzer import AnalyzerEngine

# Variable defined
ID, fake_data, used_urls, entity_columns  = {}, {}, set(), {}
COMMENT_KEYWORDS = ['comments', 'description', 'remarks', 'note', 'feedback', 'observation']

url_extensions = [
    ".com", ".net", ".org", ".edu", ".gov", ".co", ".us", ".uk", ".in", ".ru",
    ".jp", ".cn", ".de", ".fr", ".it", ".nl", ".es", ".br", ".au", ".ca",
    ".ch", ".se", ".no", ".za", ".mx", ".ar", ".be", ".kr", ".pl", ".tr",
    ".ua", ".ir", ".sa", ".ae", ".my", ".sg", ".hk", ".tw", ".nz", ".id",
    ".th", ".ph", ".vn", ".bd", ".lk", ".np", ".pk", ".cz", ".gr", ".hu",
    ".fi", ".dk", ".il", ".ie", ".pt", ".sk", ".si", ".ro", ".bg", ".rs",
    ".lt", ".lv", ".ee", ".hr", ".ba", ".md", ".ge", ".kz", ".by", ".tm",
    ".uz", ".af", ".qa", ".om", ".kw", ".bh", ".ye", ".jo", ".lb", ".sy",
    ".iq", ".ps", ".az", ".am", ".kg", ".mn", ".bt", ".mv", ".mm", ".kh",
    ".la", ".tl", ".sb", ".fj", ".pg", ".to", ".tv", ".ws", ".fm", ".ki"
]
with gzip.open("faker_data_v3.json.gz", "rt", encoding='utf-8') as f: fake_data_list = json.load(f)
# Objects
nlp = spacy.load("en_core_web_sm")
fake=Faker()
analyzer = AnalyzerEngine()

for data in fake_data_list:
    for key,value in data.items(): fake_data[key]=deque(value)
domain_pool = list(fake_data.get('url', deque()))

entity_mapping={
    'names':'PERSON',
    'emails':'EMAIL_ADDRESS',
    'phone':'PHONE_NUMBER',
    'location':'LOCATION',
    'credit':'CREDIT_CARD',
    'url':'URL',
    'country':'COUNTRY',
    'company':"ORG",
    'id':'ID',
}

mapping_file="mapping.json"
forward_mapping=defaultdict(dict)
reverse_mapping=defaultdict(dict)
comment_entity_positions = defaultdict(dict)

if os.path.exists(mapping_file):
    with open(mapping_file, "r") as f:
        mapping_data = json.load(f)
        forward_mapping.update(mapping_data.get("forward_mapping", {}))
        reverse_mapping.update(mapping_data.get("reverse_mapping", {}))

# To track time
def time_it(func):
    """Decorator to measure execution time of functions."""
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f'\n⏳ Execution time {func.__name__}: {end-start:.6f} seconds')
        return result
    return wrapper

# For comments logic
def get_comment_columns(df):
    return [col for col in df.columns if any(kw in col.lower() for kw in COMMENT_KEYWORDS)]

def mask_comment_columns(df, mapping):
    comment_cols = get_comment_columns(df)
    print('Inside mask comment cols:', comment_cols)

    for col in comment_cols:
        for i, text in df[col].astype(str).items():
            masked_text, entities = mask_comment_text(text, mapping)
            df.at[i, col] = masked_text
            if entities: comment_entity_positions[col][i] = entities
    return df

def mask_comment_text(text, mapping):
    if not text.strip(): return text, []

    doc = nlp(text)
    masked_text = text
    entities = []
    offset = 0

    for token in doc:
        if token.pos_ == "PROPN" or token.ent_type_:
            original = token.text
            fake = mapping['names'].get(original, None)

            if fake and fake != original:
                pattern = re.compile(rf"\b{re.escape(original)}(?=[\W_]|$)")
                match = pattern.search(masked_text, token.idx + offset)
                if match:
                    start, end = match.span()
                    masked_text = masked_text[:start] + fake + masked_text[end:]
                    entities.append({
                        "start": start,
                        "end": start + len(fake),
                        "original": original,
                        "fake": fake
                    })
                    offset += len(fake) - len(original)

    return masked_text, entities

def unmask_comment_columns(df, reverse_mapping, entity_metadata):
    comment_cols = get_comment_columns(df)
    print('Inside unmask comment cols:', comment_cols)

    for col in comment_cols:
        if col not in entity_metadata:
            continue

        for i, text in df[col].astype(str).items():
            if i in entity_metadata[col]:
                df.at[i, col] = unmask_comment_text(text, entity_metadata[col][i], reverse_mapping)

    return df

def unmask_comment_text(text, entities, reverse_mapping):
    for ent in sorted(entities, key=lambda x: -x['start']):
        fake = ent['fake']
        original = reverse_mapping['names'].get(fake, ent['original'])
        text = text[:ent['start']] + original + text[ent['end']:]
    return text

@time_it
def analyze_column(df):
    # Step 1: Use Presidio to analyze all columns
    comment_cols = get_comment_columns(df)

    for col in df.columns:
        if col in comment_cols: continue

        if 'id' in col.lower():
            entity_columns[col] = 'ID'
        elif 'country' in col.lower():
            entity_columns[col] = 'COUNTRY'
        else:
            unique_values = df[col].dropna().astype(str).unique()[:25]
            entity_counts = {}

            for value in unique_values:
                results = analyzer.analyze(text=value, language='en')
                for result in results:
                    entity_counts[result.entity_type] = entity_counts.get(result.entity_type, 0) + 1

            if entity_counts:
                predominant_entity = max(entity_counts, key=entity_counts.get)
                entity_columns[col] = predominant_entity
                if predominant_entity=="LOCATION":
                  org_count=0
                  for value in unique_values:
                    doc=nlp(value)
                    for ent in doc.ents:
                      if ent.label_=="ORG":
                        org_count+=1
                  if org_count>5:
                    predominant_entity="ORG"
                entity_columns[col]=predominant_entity

    # Step 2: Use SpaCy to analyze non-numeric and unclassified columns
    for col in df.select_dtypes(exclude=['number']).columns:
        if col not in entity_columns:
            unique_values = df[col].dropna().astype(str).unique()[:25]
            org_count = 0

            for value in unique_values:
                doc = nlp(value)
                for ent in doc.ents:
                    if ent.label_ == 'ORG':
                        org_count += 1

            # If more than half of the sample values are ORG, classify as ORG
            if org_count > 12:
                entity_columns[col] = 'ORG'

    return entity_columns

def modify_fake_value(category, base_fake_value, counter):
    if category == "names": return f"{base_fake_value}{string.ascii_lowercase[counter % 26]}"
    elif category == "emails":
        name, domain = base_fake_value.split("@")
        return f"{name}{counter}@{domain}"
    elif category in {"location", "country"}: return f"{base_fake_value}, District {counter % 100_000_000 + 1}"
    elif category == "url":
        # Check if the URL already exists, if so, append a country extension
        fake_value = base_fake_value
        while fake_value in used_urls:
            ext = random.choice(url_extensions)
            if not fake_value.endswith(ext):
                fake_value += ext
        used_urls.add(fake_value)
        return fake_value
    elif category == "phone": return f"{base_fake_value[:-2]}{counter % 100:02d}"
    elif category == "company": return f"{base_fake_value} Group {counter % 100_000_000 + 1}"
    elif category == "credit": return f"{base_fake_value[:-4]}{counter % 10000:04d}"
    else: return f"{base_fake_value}-{counter}"


def get_fake_value(category, original_value):
    global ID
    if original_value in forward_mapping[category]: return forward_mapping[category][original_value]
    fake_value = None
    if category == "names":
        original_tokens = str(original_value).split()
        fake_tokens = []
        used_fake_names = set(reverse_mapping[category])

        for token in original_tokens:
            if token in forward_mapping[category]:
                fake_token = forward_mapping[category][token]
            else:
                fake_token = None

                if fake_data.get(category):
                    for _ in range(len(fake_data[category])):
                        candidate = fake_data[category].popleft()
                        fake_data[category].append(candidate)

                        if candidate not in used_fake_names and candidate != token:
                            fake_token = candidate
                            break
                        else:
                            # Modify it if already used
                            counter = 0
                            base = candidate
                            while True:
                                modified = modify_fake_value(category, base, counter)
                                if modified not in used_fake_names and modified != token:
                                    fake_token = modified
                                    break
                                counter += 1
                            break

                # Save mappings
                forward_mapping[category][token] = fake_token
                reverse_mapping[category][fake_token] = token
                used_fake_names.add(fake_token)

            fake_tokens.append(fake_token)

        return " ".join(fake_tokens)


    # Special case for ID
    if category == 'id':
        length = 6
        while True:
            fake_value = fake.bothify(text=f'ID-{"#"*length}')
            if fake_value not in ID:
                ID[fake_value] = True
                break
            if len(ID) >= 10 ** length: length += 1
    elif category == 'url':
        domain1, domain2 = random.sample(domain_pool, 2)
        base_fake_value = f"https://{domain1.lower()}/{domain2.lower()}.co"

        if base_fake_value not in reverse_mapping["url"]:
            fake_value = base_fake_value
        else:
            counter = len(reverse_mapping["url"])
            fake_value = modify_fake_value("url", base_fake_value, counter)

    elif fake_data.get(category):
      length = len(fake_data[category])
      for _ in range(length):
            candidate = fake_data[category].popleft()
            fake_data[category].append(candidate)  # Reinsert at end (rotation)
            if candidate not in reverse_mapping[category]:
                fake_value = candidate
                break
      else:
          counter = len(reverse_mapping[category])
          base_fake_value = (
              random.choice(list(reverse_mapping[category]))
              if reverse_mapping[category]
              else f"{category}_"
          )
          fake_value = modify_fake_value(category, base_fake_value, counter)

    if not fake_value or fake_value in reverse_mapping[category]:
        counter = len(reverse_mapping[category])
        base = fake_value if fake_value else f"{category}_"
        fake_value = modify_fake_value(category, base, counter)

    forward_mapping[category][original_value] = fake_value
    reverse_mapping[category][fake_value] = original_value

    return fake_value

@time_it
def mask_dataframe(df):
    for col, entity in entity_columns.items():
        matching_keys = [
            key for key, value in entity_mapping.items() if value == entity
        ]
        if matching_keys:
            category = matching_keys[0]
            df[col] = df[col].astype(str).apply(
                lambda x: get_fake_value(category, x) if x else x
            )
    return df

def restore_original_value(category, fake_value):
    return reverse_mapping[category].get(fake_value, fake_value)

@time_it
def unmask_dataframe(df):
    for col, entity in entity_columns.items():
        matching_keys = [key for key, value in entity_mapping.items() if value == entity]
        if matching_keys:
            category = matching_keys[0]

            if category == "names":
                def reverse_name(val):
                    tokens = str(val).split()
                    original_tokens = [reverse_mapping[category].get(tok, tok) for tok in tokens]
                    return " ".join(original_tokens)
                df[col] = df[col].astype(str).apply(lambda x: reverse_name(x) if pd.notna(x) else x)

            else:
                df[col] = df[col].astype(str).apply(
                    lambda x: restore_original_value(category, str(x)) if pd.notna(str(x)) else x
                )
    df = unmask_comment_columns(df, reverse_mapping, comment_entity_positions)
    return df

@time_it
def compare_files(original_csv, restored_csv):
    """Compare two CSV files and return if they are identical."""
    original_df = pd.read_csv(original_csv, dtype=str).sort_index(axis=1).reset_index(drop=True)
    restored_df = pd.read_csv(restored_csv, dtype=str).sort_index(axis=1).reset_index(drop=True)

    is_identical = original_df.equals(restored_df)
    print(f"\n📊 Are files identical? {'✅ Yes' if is_identical else '❌ No'}")

    if not is_identical:
        print("⚠️ The restored file does not match the original. Investigate the mapping or masking logic.")

    return is_identical

def de_anonymize_paragraph(text):
  for category,mapping in reverse_mapping.items():
    for fake_value,original_value in mapping.items():
      if fake_value in text:
        text=text.replace(fake_value,original_value)
  return text

def save_mapping(filename):
    mapping_data={
        "filename":filename,
        "updated_at": time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()),
        "forward_mapping":forward_mapping,
        "reverse_mapping":reverse_mapping
    }
    with open(mapping_file, "a") as f:
        json.dump(mapping_data, f, indent=4)

def process_file(input_file, file_ext):
    input_base = os.path.splitext(os.path.basename(input_file))[0]
    output_dir = os.path.join(".", input_base)

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    if file_ext == ".xlsx":
        xl = pd.read_excel(input_file, sheet_name=None, dtype=str, engine='openpyxl')

        for sheet_name, df in xl.items():
            print(f"\n🔍 Processing sheet: {sheet_name}")

            original_csv = os.path.join(output_dir, f"{sheet_name}_original.csv")
            masked_csv = os.path.join(output_dir, f"{sheet_name}_masked.csv")
            restored_csv = os.path.join(output_dir, f"{sheet_name}_restored.csv")

            df.to_csv(original_csv, index=False)

            entity_columns = analyze_column(df)
            print(f"Detected entities: {entity_columns}")

            masked_df = mask_dataframe(df.copy())
            masked_df = mask_comment_columns(masked_df, forward_mapping)
            restored_df = unmask_dataframe(masked_df.copy())

            masked_df.to_csv(masked_csv, index=False)
            restored_df.to_csv(restored_csv, index=False)

            compare_files(original_csv, restored_csv)

            save_mapping(f'{input_file}-->{sheet_name}')
            entity_columns.clear()


    elif file_ext == ".csv":
        df = pd.read_csv(input_file, dtype=str, low_memory=False)
        print(f"\n🔍 Processing CSV: {input_file}")

        original_csv = os.path.join(output_dir, "original.csv")
        masked_csv = os.path.join(output_dir, "anonymized.csv")
        restored_csv = os.path.join(output_dir, "restored.csv")

        if not os.path.exists(output_dir): os.makedirs(output_dir)

        df.to_csv(original_csv, index=False)

        entity_columns = analyze_column(df)
        print(f"Detected entities: {entity_columns}")

        masked_df = mask_dataframe(df.copy())
        masked_df = mask_comment_columns(masked_df, forward_mapping)
        restored_df = unmask_dataframe(masked_df.copy())

        masked_df.to_csv(masked_csv, index=False)
        restored_df.to_csv(restored_csv, index=False)

        compare_files(original_csv, restored_csv)

        save_mapping(input_file)
    else:
        raise ValueError("Unsupported file type. Use .csv or .xlsx only.")

if __name__ == "__main__":
    input_file = "smaller_30k_companies.csv"
    file_ext = os.path.splitext(input_file)[-1].lower()
    process_file(input_file, file_ext)


🔍 Processing CSV: smaller_30k_companies.csv

⏳ Execution time analyze_column: 6.245137 seconds
Detected entities: {'id': 'ID', 'name': 'ORG', 'domain': 'URL', 'size range': 'DATE_TIME', 'locality': 'LOCATION', 'country': 'COUNTRY', 'linkedin url': 'URL', 'current employee estimate': 'US_DRIVER_LICENSE', 'total employee estimate': 'US_DRIVER_LICENSE', 'names': 'PERSON'}

⏳ Execution time mask_dataframe: 2.282188 seconds
Inside mask comment cols: []
Inside unmask comment cols: []

⏳ Execution time unmask_dataframe: 0.207653 seconds

📊 Are files identical? ✅ Yes

⏳ Execution time compare_files: 0.414886 seconds


In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")

doc = nlp(" debugged energy grid anomalies.")

for token in doc:
  print(token.text, token.pos_)
  if token.pos_ == "PROPN" or token.ent_type_:
      print(token.text)
      # print(reverse_mapping['names'].get(token.text))


Rem PROPN
Rem
debugged VERB
energy NOUN
grid NOUN
anomalies NOUN
. PUNCT


In [ ]:
# Seeing the differences b/w codes
import pandas as pd

def difference_in_csv_sm(path):
    original_df = pd.read_csv(path)
    restored_df = pd.read_csv('./smaller_30k_companies/restored.csv')
    diff_df = original_df.compare(restored_df, keep_equal=False)

    print(diff_df.head(10))
    diff_df.to_csv('difference.csv', index=False)

def difference_in_csv(path):
    original_df = pd.read_csv(path)
    restored_df = pd.read_csv('./for_colab_test/restored.csv')
    anonymized_df = pd.read_csv('./for_colab_test/anonymized.csv')

    diff_df = original_df.compare(restored_df, keep_equal=False)

    diff_df.columns = [
        (col[0], 'original' if col[1] == 'self' else 'restored')
        for col in diff_df.columns
    ]
    diff_df.columns = pd.MultiIndex.from_tuples(diff_df.columns)

    diff_df = diff_df.reset_index()

    anonymized_rows = anonymized_df.loc[diff_df['index']]
    anonymized_rows = anonymized_rows.reset_index(drop=True)
    diff_df = diff_df.drop(columns='index')

    # Flatten multi-index columns for original/restored
    diff_flat = pd.DataFrame()
    for col in diff_df.columns.get_level_values(0).unique():
        diff_flat[f'{col} (original)'] = diff_df[col]['original']
        diff_flat[f'{col} (restored)'] = diff_df[col]['restored']

    for col in anonymized_rows.columns:
        if 'comment' in col.lower():
            diff_flat[f'{col} (anonymized)'] = anonymized_rows[col]

    diff_flat.to_csv('differences.csv', index=False)
    print(diff_flat.head(10))

def difference_in_excel(path):
  original_df = pd.read_excel(path)
  restored_df = pd.read_excel('restored.xlsx')
  diff_df = original_df.compare(restored_df, keep_equal=False)
  print(diff_df.head(10))
  diff_df.to_excel('differences.xlsx')

def difference_in_parquet(path):
  original_df = pd.read_parquet(path)
  restored_df = pd.read_parquet('Sheet1_restored.parquet')
  diff_df = original_df.compare(restored_df, keep_equal=False)
  print(diff_df.head(10))
  diff_df.to_parquet('differences.parquet')

difference_in_csv_sm('smaller_30k_companies.csv')
# difference_in_csv('for_colab_test.csv')
# difference_in_excel()
# difference_in_parquet()

Empty DataFrame
Columns: []
Index: []


In [ ]:
# Testing out fuzzy match
import nltk

nltk.download('stopwords')

text = '''Ford-Mendez Corporation Inc., founded in 1911, operates in the information
technology and services industry. Based in Port Brett, Delaware Region, Afghanistan,
the company has over 10,000 employees and a current employee estimate of 274,047.
The total employee estimate is 716,906. For more details, you can visit their LinkedIn page here.
Additionally, a key employee at the company, A Aaban Aabarana, has been working there for several years.
Kelly, Hobbs and Garcia Industries International, established in 1968,
is also a major player in the information technology and services sector. Located in Jefferyburgh,
South Dakota, Albania, this company too has over 10,000 employees, with a current employee estimate
of 190,771 and a total employee estimate of 341,369. You can learn more about the company by visiting
their LinkedIn profile here. One of their top executives, Aabarna Aabarniga, has contributed greatly to
their success.
'''

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Isolating verbs and adjectives from text
import re
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

common_verbs = {
    'is', 'was', 'are', 'were', 'be', 'being', 'been', 'have', 'has', 'had',
    'get', 'gets', 'got', 'do', 'does', 'did', 'can', 'could', 'will', 'would',
    'shall', 'should', 'may', 'might', 'must', 'found', 'run', 'operate', 'used',
    'work', 'working', 'known', 'help', 'helped', 'helping', 'contributed', 'success', 'successful',
    'unsuccessful'
}

def get_entity_candidates(text):
    words = re.findall(r'\b\w+\b', text)
    filtered = [
        word for word in words
        if word.lower() not in stop_words
        and word.lower() not in common_verbs
        and len(word) > 2
        # and not word.isdigit()
    ]
    return filtered

corporus = "Jams Bond was found operating a project called EnrgyNet in New Yrk in 2023. It helps clients."
print(get_entity_candidates(corporus))


['Jams', 'Bond', 'operating', 'project', 'called', 'EnrgyNet', 'New', 'Yrk', '2023', 'helps', 'clients']


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
!pip install rapidfuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 24.1 MB/s eta 0:00:00


In [ ]:
# Testing out matching using fuzzy
import re

verb_roots = {'be', 'is', 'have', 'do', 'say', 'go', 'can', 'get', 'make', 'know', 'think', 'take', 'see', 'come', 'want', 'look', 'use', 'find', 'give', 'tell', 'work', 'call', 'try', 'ask', 'need', 'feel', 'become', 'leave', 'put', 'mean', 'keep', 'let', 'like', 'help'}
adj_roots = {'good', 'manag', 'new', 'first', 'last', 'long', 'great', 'little', 'own', 'other', 'old', 'right', 'big', 'high', 'different', 'small', 'large', 'next', 'early', 'young', 'important'}
adv_roots = {'very', 'too', 'really', 'just', 'quite', 'soon', 'again', 'still', 'always', 'often'}
prepositions = {'in', 'on', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'of', 'off', 'a', 'the', 'too', 'and', 'or', 'what', 'else', 'if'}
punctuations = {'.', ',', ';', ':', '!', '?', '(', ')', '[', ']', '{', '}', '"', "'"}
common_words = {'father', 'mother', 'daughter', 'son', 'aunty', 'aunt', 'uncle', 'sister', 'brother', }

def is_excluded(word):
    w = word.lower()
    return (
        any(re.fullmatch(rf"{root}(s|ed|ing)?", w) for root in verb_roots) or
        any(re.fullmatch(rf"{root}(er|est|ly)?", w) for root in adj_roots) or
        any(re.fullmatch(rf"{root}(ly|ing)?", w) for root in adv_roots) or
        any(re.fullmatch(rf"{root}(s|'s|ed|ly|ing)?", w) for root in common_words) or
        w in prepositions or
        w in punctuations
    )


In [ ]:
from rapidfuzz import process, fuzz

def correct_word(word, valid_names, threshold=65):
    match = process.extractOne(word, valid_names, scorer=fuzz.ratio)
    return match[0] if match and match[1] >= threshold else None

@time_it
def isolate_and_correct_entities(text, valid_names):
    tokens = text.split()
    entities = []

    for idx, word in enumerate(tokens):
        if not is_excluded(word):
            corrected = correct_word(word, valid_names)
            if corrected:
                entities.append((idx, word, corrected.capitalize()))

    return entities


In [ ]:
text = """Theressa is a manager working for Wiliams. Ruthu Jenkinz is the mother of
  Fransis Waner and Kaitlyn Meyers."""

valid_names = [name.lower() for name in forward_mapping['names']]
entities = isolate_and_correct_entities(text, valid_names)

for idx, original, corrected in entities:
    print(f"Position {idx}: '{original}' corrected to '{corrected}'")

tokens = text.split()
for idx, original, corrected in entities:
    tokens[idx] = corrected

corrected_text = ' '.join(tokens)

print("Corrected text:")
print(corrected_text)


⏳ Execution time isolate_and_correct_entities: 0.007842 seconds
Position 0: 'Theressa' corrected to 'Theresa'
Position 6: 'Wiliams.' corrected to 'Williams'
Position 7: 'Ruthu' corrected to 'Ruth'
Position 8: 'Jenkinz' corrected to 'Jenkins'
Position 13: 'Fransis' corrected to 'Francis'
Position 14: 'Waner' corrected to 'Wagner'
Position 16: 'Kaitlyn' corrected to 'Kaitlyn'
Position 17: 'Meyers.' corrected to 'Meyers'
Corrected text:
Theresa is a manager working for Williams Ruth Jenkins is the mother of Francis Wagner and Kaitlyn Meyers


In [ ]:
import time
from rapidfuzz import process, fuzz

def correct_word(word, valid_names, threshold=65):
    name_match = process.extractOne(word, valid_names, scorer=fuzz.ratio)
    if name_match and name_match[1] >= threshold: return name_match[0]
    return None


start_time = time.time()
f_mapping_names = [key.lower() for key in forward_mapping['names']]
end_time = time.time()

print(f'Mapping list time: {end_time-start_time}')
text = """Theressa is a manager working for Wiliams Ruthj Jenkinz Fransis Waner Kaitlyn Meyers Nicolas Stevans
Dona Berd Solushunz Stefanie Miller.""".split()

start_time = time.time()
results = [correct_word(word, f_mapping_names) for word in text]
end_time = time.time()
print(f'Result: {end_time-start_time}')

print(results)

['Alexandra', 'Alice', 'Armstrong', 'Arthur', 'Ariel', 'Alex', 'Ashley', 'Alejandro', 'Antonio', 'Andrew']
Mapping list time: 0.0009248256683349609
Result: 0.003665447235107422
['theresa', 'lisa', None, 'maynard', 'wong', 'ford', 'williams', 'ruth', 'jenkins', 'francis', 'wagner', 'kaitlyn', 'meyers', 'nicolas', 'evans', 'donna', 'beard', None, 'stefanie', 'miller']


In [ ]:
# Testing entity isolation and correction
import spacy
import re
from rapidfuzz import process, fuzz

# Load spaCy small English model (you can also try 'en_core_web_md' if you want better tagging)
nlp = spacy.load('en_core_web_sm')

verb_roots = {'be', 'is', 'have', 'do', 'say', 'go', 'can', 'get', 'make', 'know', 'think', 'take', 'see', 'come', 'want', 'look', 'use', 'find', 'give', 'tell', 'work', 'call', 'try', 'ask', 'need', 'feel', 'become', 'leave', 'put', 'mean', 'keep', 'let', 'like', 'help'}
adj_roots = {'good', 'manag', 'new', 'first', 'last', 'long', 'great', 'little', 'own', 'other', 'old', 'right', 'big', 'high', 'different', 'small', 'large', 'next', 'early', 'young', 'important'}
adv_roots = {'very', 'too', 'really', 'just', 'quite', 'soon', 'again', 'still', 'always', 'often'}
prepositions = {'in', 'on', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'of', 'off', 'a', 'the', 'too', 'and', 'or', 'what', 'else', 'if'}
punctuations = {'.', ',', ';', ':', '!', '?', '(', ')', '[', ']', '{', '}', '"', "'"}
common_words = {'father', 'mother', 'daughter', 'son', 'aunty', 'aunt', 'uncle', 'sister', 'brother'}

def is_excluded(word):
    w = word.lower()
    return (
        any(re.fullmatch(rf"{root}(s|ed|ing)?", w) for root in verb_roots) or
        any(re.fullmatch(rf"{root}(er|est|ly)?", w) for root in adj_roots) or
        any(re.fullmatch(rf"{root}(ly|ing)?", w) for root in adv_roots) or
        any(re.fullmatch(rf"{root}(s|'s|ed|ly|ing)?", w) for root in common_words) or
        w in prepositions or
        w in punctuations
    )

def correct_word(word, valid_names, threshold=65):
    match = process.extractOne(word, valid_names, scorer=fuzz.ratio)
    return match[0] if match and match[1] >= threshold else None

def isolate_and_correct_entities_spacy(text, valid_names, threshold=65):
    doc = nlp(text)
    tokens = [token.text for token in doc]
    corrected_tokens = tokens.copy()

    for i, token in enumerate(doc):
        if token.is_punct or token.is_space:
            continue

        if token.pos_ in {"NOUN", "PROPN"}:
            word = token.text
            if not is_excluded(word):
                corrected = correct_word(word, valid_names, threshold)
                if corrected:
                    corrected_tokens[i] = corrected.capitalize()

    corrected_text = ''.join([
        token if token in punctuations or token.isspace() else ' ' + token
        for token in corrected_tokens
    ]).strip()

    return corrected_text

# --- Example usage:

text = """Theressa is a manager working for Wiliams. Ruthu Jenkinz is the mother of Fransis Waner and Kaitlyn Meyers."""

# Assume forward_mapping['names'] already loaded somewhere
valid_names = [name.lower() for name in forward_mapping['names']]

corrected_text = isolate_and_correct_entities_spacy(text, valid_names)
print(corrected_text)


Original: Nw York, Corrected: New York, Position: (15, 22), Type: GPE


In [3]:
!pip install gliner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 71.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46

In [ ]:
from gliner import GLiNER
from rapidfuzz import process, fuzz

model = GLiNER.from_pretrained("urchade/gliner_base")

def correct_word(word, valid_list, threshold=65):
    match = process.extractOne(word, valid_list, scorer=fuzz.ratio)
    return match[0] if match and match[1] >= threshold else word  # fallback if no good match

def find_and_correct_entities(text, valid_names, valid_company):
    entities = model.predict_entities(text, labels=["person", "organization"])

    corrected_entities = []

    for ent in entities:
        entity_text = ent['text']
        start = ent['start']
        end = ent['end']
        label = ent['label']

        if label == "person":
            tokens = entity_text.split()
            corrected_tokens = [
                correct_word(token.lower(), valid_names).capitalize()
                for token in tokens
            ]
            corrected = " ".join(corrected_tokens)

        elif label == "organization":
            corrected = correct_word(entity_text.lower(), valid_company).capitalize()

        else: corrected = entity_text

        corrected_entities.append({
            'original': entity_text,
            'corrected': corrected,
            'start': start,
            'end': end,
            'label': label
        })

    return corrected_entities

def replace_entities_in_text(text, entities):
    entities = sorted(entities, key=lambda x: x['start'], reverse=True)

    for ent in entities:
        text = text[:ent['start']] + ent['corrected'] + text[ent['end']:]
    return text

@time_it
def main():
  text = """Theressa is a manager working for Wiliams. Ruthu Jenkinz is the mother of Fransis Waner and Kaitlyy Meeeers.
  Donaa Gavln hasn't been seen for a while now by Nickol. THis is concerning!"""
  valid_names = [name.lower() for name in forward_mapping['names']]  # should be tokens
  valid_company = [company.lower() for company in forward_mapping['company']]

  entities = find_and_correct_entities(text, valid_names, valid_company)
  corrected_text = replace_entities_in_text(text, entities)

  print("\nCorrected Text:\n", corrected_text)

main()


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



Corrected Text:
 Theresa is a manager working for Williams. Ruth Jenkins is the mother of Francis Wagner and Kaitlyn Meyers.
  Donna Gavin hasn't been seen for a while now by Nikon. THis is concerning!

⏳ Execution time main: 0.805174 seconds

Corrected Text:
 Theresa is a manager working for Williams. Ruth Jenkins is the mother of Francis Wagner and Kaitlyn Meyers.
  Donna Gavin hasn't been seen for a while now by Nikon. THis is concerning!

⏳ Execution time main: 0.640440 seconds

Corrected Text:
 Theresa is a manager working for Williams. Ruth Jenkins is the mother of Francis Wagner and Kaitlyn Meyers.
  Donna Gavin hasn't been seen for a while now by Nikon. THis is concerning!

⏳ Execution time main: 0.651691 seconds

Corrected Text:
 Theresa is a manager working for Williams. Ruth Jenkins is the mother of Francis Wagner and Kaitlyn Meyers.
  Donna Gavin hasn't been seen for a while now by Nikon. THis is concerning!

⏳ Execution time main: 0.645857 seconds

Corrected Text:
 Theres

In [7]:
from gliner import GLiNER
from rapidfuzz import process, fuzz
from faker import Faker
import json
import time

model = GLiNER.from_pretrained("urchade/gliner_base")
faker = Faker()

forward_mapping = {
    "names": {
        "Theresa": "Aanik",
        "Fransis": "Ben",
        "Kaitlyn": "Lora",
        "Nickol": "Chris",
        "Donaa": "Alia",
        "Ruthu": "Fay"
    },
    "company": {
        "Williams-Waller Co": "ibm",
        "Tata Consultancy Services": "tcs",
        "IBM": "oracle"
    }
}

def time_it(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f"\nExecution time: {end - start:.4f} seconds")
        return result
    return wrapper

def correct_with_forward_mapping(word, mapping_dict, threshold=85):
    match = process.extractOne(word, mapping_dict.keys(), scorer=fuzz.ratio)
    if match and match[1] >= threshold:
        return match[0], mapping_dict[match[0]], False
    return word, None, True  # new word

def find_and_correct_entities(text):
    entities = model.predict_entities(text, labels=["person", "organization"])
    corrected_entities = []

    new_entity_mappings = {
        "forward_mapping": {},
        "reverse_mapping": {}
    }

    for ent in entities:
        entity_text = ent['text']
        start = ent['start']
        end = ent['end']
        label = ent['label']

        if label == "person":
            tokens = entity_text.split()
            corrected_tokens = []
            for token in tokens:
                original_token = token.strip()
                mapped_key, mapped_value, is_new = correct_with_forward_mapping(original_token, forward_mapping['names'])
                if is_new:
                    fake_name = faker.first_name()
                    new_entity_mappings["forward_mapping"][original_token] = fake_name
                    new_entity_mappings["reverse_mapping"][fake_name] = original_token
                    corrected_tokens.append(mapped_key)
                else:
                    corrected_tokens.append(mapped_key)
            corrected = " ".join(corrected_tokens)

        elif label == "organization":
            mapped_key, mapped_value, is_new = correct_with_forward_mapping(entity_text, forward_mapping['company'])
            if is_new:
                fake_company = faker.company()
                new_entity_mappings["forward_mapping"][entity_text] = fake_company
                new_entity_mappings["reverse_mapping"][fake_company] = entity_text
                corrected = mapped_key
            else:
                corrected = mapped_key

        else:
            corrected = entity_text  # not person/org

        corrected_entities.append({
            'original': entity_text,
            'corrected': corrected,
            'start': start,
            'end': end,
            'label': label
        })

    return corrected_entities, new_entity_mappings

def replace_entities_in_text(text, entities):
    entities = sorted(entities, key=lambda x: x['start'], reverse=True)
    for ent in entities:
        text = text[:ent['start']] + ent['corrected'] + text[ent['end']:]
    return text

@time_it
def main():
    text = """Theressa is a manager working for Wiliams. Ruthu Jenkinz is the mother of Fransis Waner and Kaitlyy Meeeers.
    Donaa Gavln hasn't been seen for a while now by Nickol. THis is concerning!"""

    entities, new_entity_mappings = find_and_correct_entities(text)
    corrected_text = replace_entities_in_text(text, entities)

    print("\nCorrected Text:\n", corrected_text)

    print("\nOld Entity Mappings:\n", json.dumps(forward_mapping, indent=2))
    print("\nNew Entity Mappings:\n", json.dumps(new_entity_mappings, indent=2))

main()


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



Corrected Text:
 Theresa is a manager working for Wiliams. Ruthu Jenkinz is the mother of Fransis Waner and Kaitlyn Meeeers.
    Donaa Gavln hasn't been seen for a while now by Nickol. THis is concerning!

Old Entity Mappings:
 {
  "names": {
    "Theresa": "Aanik",
    "Fransis": "Ben",
    "Kaitlyn": "Lora",
    "Nickol": "Chris",
    "Donaa": "Alia",
    "Ruthu": "Fay"
  },
  "company": {
    "Williams-Waller Co": "ibm",
    "Tata Consultancy Services": "tcs",
    "IBM": "oracle"
  }
}

New Entity Mappings:
 {
  "forward_mapping": {
    "Wiliams": "Lambert Group",
    "Jenkinz": "Jessica",
    "Waner": "Andrew",
    "Meeeers": "Alex",
    "Gavln": "Melinda",
    "Nickol": "Moore, Freeman and Moore"
  },
  "reverse_mapping": {
    "Lambert Group": "Wiliams",
    "Jessica": "Jenkinz",
    "Andrew": "Waner",
    "Alex": "Meeeers",
    "Melinda": "Gavln",
    "Moore, Freeman and Moore": "Nickol"
  }
}

Execution time: 1.1870 seconds
